# **<u>Proyecto de automatización de trabajo: iteración sobre Sheets en carpetas de Google Drive, append en otra Sheets de BBDD consolidada, y exportación a BigQuery.</u>**

---

## <span style="background-color: #FFC0CB">Pending:</span>
            1. establecer un trigger semanal
            2. documentación READme [paso a paso proyecto]
                      
          

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Importación de bibliotecas y montaje de drive

In [2]:
import os
import time
import pandas as pd
from google.colab import drive
import gspread
import pytz
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from oauth2client.client import GoogleCredentials
from datetime import datetime
from google.cloud import bigquery
from googleapiclient.discovery import build
from collections import Counter

In [ ]:
# pip install pytz

## Tranformar la zona horaria a Madrid con la biblioteca pytz


In [3]:
madrid_tz = pytz.timezone('Europe/Madrid')

# Obtiene la hora actual en la zona horaria de Madrid
madrid_time = datetime.now(madrid_tz)

print("Hora actual en Madrid:", madrid_time)

Hora actual en Madrid: 2025-07-31 13:08:28.133067+02:00


## Inicialización de credenciales de Google, Google API y archivo JSON

### **Atención**: previamente se debe crear un proyecto en la nube y seguir una serie de pasos para autenticar el acceso a sus archivos de drive. Más información [aquí](https://cloud.google.com/gcp/?hl=es&utm_source=bing&utm_medium=cpc&utm_campaign=emea-es-all-es-bkws-all-all-trial-e-gcp-1011340&utm_content=text-ad-none-any-DEV_c-CRE_-ADGP_Hybrid+%7C+BKWS+-+EXA+%7C+Txt+-+GCP+-+General+-+v3-KWID_43700061311461132-kwd-77240890777560:loc-170-userloc_164782&utm_term=KW_google%20cloud-NET_o-PLAC_&&refclickid=NNE_HotelDigitalMediaCampaign&pmed=DPM_CVG_TRV_HOT_NNE_SEM_GOG_&gclid=e13bffff37a5160b7e7e99c1ad5ecd04&gclsrc=3p.ds&+)


In [4]:
# Definir el ámbito de actuación y credenciales
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/spreadsheets']

creds = ServiceAccountCredentials.from_json_keyfile_name('/content/drive/Shareddrives/(02 MAD) Iniciativas - AAPP - JP/Automatización_py/test-automation-gsheets-5fa673850271.json', scope)

# Autenticarse con Google Sheets API
client = gspread.authorize(creds)
drive_service = build('drive', 'v3', credentials=creds)

# Acceder a la hoja de cálculo unificada

In [5]:
# Acceder a la hoja de cálculo unificada
sheet_unificado = client.open_by_key('1T8l8omO07iWkUyB2tuc6wGf_RJhsgifWUgTb6L2LPN8')

### Pre-procesamiento de datos:

* Nulos
* Columnas a descartar

In [6]:
# Acceder a la hoja de cálculo
sheet = client.open_by_key('1T8l8omO07iWkUyB2tuc6wGf_RJhsgifWUgTb6L2LPN8')  # La llave de entrada

from collections import Counter

def obtener_estadisticas_y_filas(worksheet):
    datos = worksheet.get_all_values()  # Obtiene todos los valores como lista de listas

    if not datos or len(datos) < 2:  # Si no hay datos o solo hay encabezados
        print(f"\nHoja: {worksheet.title} - Vacía")
        return

    num_filas = len(datos) - 1  # Resto 1 para excluir la fila de encabezado

    encabezados = datos[0]  # Primera fila como encabezados
    filas = datos[1:]  # Datos sin encabezados

    contador = Counter(encabezados)
    nombres_unicos = []
    usados = {}

    for col in encabezados:
        if contador[col] > 1:
            usados[col] = usados.get(col, 0) + 1
            nuevo_nombre = f"{col}_{usados[col]}"
        else:
            nuevo_nombre = col
        nombres_unicos.append(nuevo_nombre)

    # Convertir a lista de diccionarios con encabezados únicos
    records = [dict(zip(nombres_unicos, fila)) for fila in filas]

    # Extraer valores numéricos de todas las columnas
    valores_numericos = [value for row in records for value in row.values() if isinstance(value, (int, float))]

    maximo = max(valores_numericos) if valores_numericos else "No hay datos numéricos"
    minimo = min(valores_numericos) if valores_numericos else "No hay datos numéricos"

    celdas_totales = num_filas * len(nombres_unicos)
    celdas_vacias = sum([list(row.values()).count('') for row in records])

    print(f"\nHoja: {worksheet.title}")
    print(f"Número de filas (sin encabezado): {num_filas}") #Multiv
    # print(f"Máximo: {maximo}, Mínimo: {minimo}")
    print(f"Celdas vacías: {celdas_vacias} de {celdas_totales} celdas totales")

# Bucle para recorrer hojas 0, 1 y 2 [Boletines autonómicos, parlamentarios y agendas]
for i in range(3):  # Hojas de índice 0, 1 y 2
    worksheet = sheet.get_worksheet(i)
    obtener_estadisticas_y_filas(worksheet)



Hoja: Boletines autonómicos
Número de filas (sin encabezado): 10991
Celdas vacías: 59499 de 175856 celdas totales

Hoja: Boletines parlamentarios
Número de filas (sin encabezado): 18210
Celdas vacías: 79778 de 236730 celdas totales

Hoja: Agendas parlamentarias
Número de filas (sin encabezado): 4115
Celdas vacías: 15674 de 57610 celdas totales


# IDs de las carpetas

*   Boletines parlamentarios
*   Boletines autonómicos
*   Agendas parlamentarias





In [7]:
# IDs de carpetas correctos (obtenidos del diagnóstico)
FOLDER_IDS = {
    'parlamentarios': '1tCPZdP81pM2sTuGMAFUYsvGNi45lnW8E',
    'autonomicos': '1Lnp4jcdASi72QedN6HPviE2R6Gn8td2X',
    'agendas': '1CCdTfcadG4MadY5Tp665yzhEBAzPQ1Km'
}

# Obtener todas las hojas de Google Sheets en una carpeta específica que hayan sido modificadas después de una fecha de corte

In [8]:
# Configuración
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive',
         'https://www.googleapis.com/auth/spreadsheets']

creds = ServiceAccountCredentials.from_json_keyfile_name(
    '/content/drive/Shareddrives/(02 MAD) Iniciativas - AAPP - JP/Automatización_py/test-automation-gsheets-5fa673850271.json',
    scope
)

client = gspread.authorize(creds)
drive_service = build('drive', 'v3', credentials=creds)

def verificar_credenciales():
    """Verifica que las credenciales funcionen correctamente"""
    print("VERIFICANDO CREDENCIALES")
    print("=" * 60)

    try:
        about = drive_service.about().get(fields='user').execute()
        print(f" Autenticación exitosa")
        print(f" Email: {creds.service_account_email}")
        print(f" Usuario: {about.get('user', {}).get('displayName', 'N/A')}")
        return True
    except Exception as e:
        print(f" Error en credenciales: {e}")
        return False

def verificar_shared_drives():
    """Verifica acceso a Shared Drives (Team Drives)"""
    print("\n VERIFICANDO SHARED DRIVES")
    print("=" * 60)

    try:
        # Listar shared drives
        drives = drive_service.drives().list().execute()
        shared_drives = drives.get('drives', [])

        if not shared_drives:
            print(" No se encontraron Shared Drives accesibles")
            return []

        print(f" Shared Drives encontrados: {len(shared_drives)}")

        drives_info = []
        for drive in shared_drives:
            print(f"\n {drive['name']}")
            print(f"    ID: {drive['id']}")
            print(f"    Creado: {drive.get('createdTime', 'N/A')}")

            drives_info.append({
                'id': drive['id'],
                'name': drive['name']
            })

            # Buscar carpetas en este shared drive
            print(f"    Buscando carpetas en este drive...")
            try:
                query = f"'{drive['id']}' in parents and mimeType='application/vnd.google-apps.folder' and trashed=false"
                results = drive_service.files().list(
                    q=query,
                    fields="files(id, name)",
                    pageSize=10,
                    supportsAllDrives=True,
                    includeItemsFromAllDrives=True
                ).execute()

                carpetas = results.get('files', [])
                print(f"    Carpetas encontradas: {len(carpetas)}")

                for carpeta in carpetas:
                    print(f"       {carpeta['name']} (ID: {carpeta['id']})")

            except Exception as e:
                print(f"    Error al buscar carpetas: {e}")

        return drives_info

    except Exception as e:
        print(f" Error al verificar shared drives: {e}")
        return []

def buscar_en_todos_los_drives():
    """Busca carpetas en todos los drives (My Drive + Shared Drives)"""
    print("\n BÚSQUEDA EN TODOS LOS DRIVES")
    print("=" * 60)

    keywords = ['Boletines', 'Parlamentarios', 'Autonómicos', 'Agendas']

    for keyword in keywords:
        print(f"\n Buscando '{keyword}'...")

        try:
            # Buscar en todos los drives
            query = f"name contains '{keyword}' and mimeType='application/vnd.google-apps.folder' and trashed=false"

            results = drive_service.files().list(
                q=query,
                fields="files(id, name, parents, driveId, owners)",
                pageSize=20,
                supportsAllDrives=True,
                includeItemsFromAllDrives=True
            ).execute()

            carpetas = results.get('files', [])

            if carpetas:
                print(f" Encontradas {len(carpetas)} carpetas con '{keyword}':")
                for carpeta in carpetas:
                    print(f"    {carpeta['name']}")
                    print(f"       ID: {carpeta['id']}")
                    print(f"       Drive ID: {carpeta.get('driveId', 'My Drive')}")
                    print(f"       Propietarios: {len(carpeta.get('owners', []))}")
                    print()
            else:
                print(f" No se encontraron carpetas con '{keyword}'")

        except Exception as e:
            print(f" Error buscando '{keyword}': {e}")

def probar_acceso_carpeta_especifica():
    """Prueba acceso directo a la carpeta específica"""
    print("\n PROBANDO ACCESO A CARPETA ESPECÍFICA")
    print("=" * 60)

    folder_id = "1tCPZdP81pM2sTuGMAFUYsvGNi45lnW8E"

    # Esperar un poco por si los permisos necesitan tiempo
    print(" Esperando 5 segundos para propagación de permisos...")
    time.sleep(5)

    try:
        # Intentar acceso con supportsAllDrives=True
        folder = drive_service.files().get(
            fileId=folder_id,
            supportsAllDrives=True
        ).execute()

        print(f" ÉXITO: Carpeta encontrada!")
        print(f" Nombre: {folder.get('name')}")
        print(f" ID: {folder.get('id')}")
        print(f" Drive ID: {folder.get('driveId', 'My Drive')}")
        print(f" Modificada: {folder.get('modifiedTime')}")

        # Intentar listar archivos
        print(f"\n📄 Listando archivos en la carpeta...")
        query = f"'{folder_id}' in parents and trashed=false"

        results = drive_service.files().list(
            q=query,
            fields="files(id, name, mimeType, modifiedTime)",
            pageSize=10,
            supportsAllDrives=True,
            includeItemsFromAllDrives=True
        ).execute()

        archivos = results.get('files', [])
        print(f" Archivos encontrados: {len(archivos)}")

        if archivos:
            sheets_count = 0
            for archivo in archivos:
                tipo = ""
                if archivo['mimeType'] == 'application/vnd.google-apps.spreadsheet':
                    tipo = " Google Sheets"
                    sheets_count += 1
                elif archivo['mimeType'] == 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet':
                    tipo = " Excel"
                else:
                    tipo = " Otro"

                print(f"   {tipo} {archivo['name']}")
                print(f"       ID: {archivo['id']}")
                print(f"       Modificado: {archivo.get('modifiedTime')}")
                print()

            print(f"📊 Total Google Sheets encontrados: {sheets_count}")

        return True

    except Exception as e:
        print(f" Error al acceder a la carpeta: {e}")
        return False

def verificar_ruta_local():
    """Verifica las carpetas locales montadas"""
    print("\n VERIFICANDO CARPETAS LOCALES MONTADAS")
    print("=" * 60)

    rutas_verificar = [
        '/content/drive/Shareddrives/(02 MAD) Iniciativas - AAPP - JP/Boletines Parlamentarios',
        '/content/drive/Shareddrives/(02 MAD) Iniciativas - AAPP - JP/Boletines Autonómicos',
        '/content/drive/Shareddrives/(02 MAD) Iniciativas - AAPP - JP/Agendas parlamentarias'
    ]

    for ruta in rutas_verificar:
        print(f"\n Verificando: {ruta}")

        if os.path.exists(ruta):
            print(f"    Carpeta existe localmente")

            # Listar archivos
            try:
                archivos = os.listdir(ruta)
                print(f"    Archivos encontrados: {len(archivos)}")

                # Mostrar algunos archivos
                for archivo in archivos[:5]:
                    print(f"       {archivo}")

                if len(archivos) > 5:
                    print(f"      ... y {len(archivos) - 5} más")

            except Exception as e:
                print(f"    Error al listar archivos: {e}")
        else:
            print(f"    Carpeta no existe localmente")

def main():
    """Función principal de diagnóstico avanzado"""
    print(" DIAGNÓSTICO AVANZADO DE ACCESO A CARPETAS")
    print(f" Hora actual: {datetime.now()}")
    print("=" * 80)

    # 1. Verificar credenciales
    if not verificar_credenciales():
        print(" Las credenciales fallan. Revisar archivo de service account.")
        return

    # 2. Verificar shared drives
    shared_drives = verificar_shared_drives()

    # 3. Búsqueda exhaustiva
    buscar_en_todos_los_drives()

    # 4. Probar acceso específico
    acceso_exitoso = probar_acceso_carpeta_especifica()

    # 5. Verificar carpetas locales
    verificar_ruta_local()

    # 6. Recomendaciones
    print("\n💡 RECOMENDACIONES:")
    print("=" * 60)

    if acceso_exitoso:
        print(" SOLUCIÓN ENCONTRADA: La carpeta es accesible")
        print(" Puedes usar el código original con estos parámetros:")
        print("   - supportsAllDrives=True")
        print("   - includeItemsFromAllDrives=True")
    else:
        print(" PROBLEMA PERSISTENTE:")
        print("1. Verifica que compartiste la carpeta correcta")
        print("2. Asegúrate de que los permisos sean 'Editor' o 'Gestor de contenido'")
        print("3. Espera 5-10 minutos para propagación de permisos")
        print("4. Considera crear carpetas nuevas en Google Drive nativo")

    print("\n DIAGNÓSTICO COMPLETADO")

if __name__ == "__main__":
    main()

 DIAGNÓSTICO AVANZADO DE ACCESO A CARPETAS
 Hora actual: 2025-07-31 11:08:46.261841
VERIFICANDO CREDENCIALES
 Autenticación exitosa
 Email: test-automation-sheets@test-automation-gsheets.iam.gserviceaccount.com
 Usuario: test-automation-sheets@test-automation-gsheets.iam.gserviceaccount.com

 VERIFICANDO SHARED DRIVES
 No se encontraron Shared Drives accesibles

 BÚSQUEDA EN TODOS LOS DRIVES

 Buscando 'Boletines'...
 Encontradas 2 carpetas con 'Boletines':
    Boletines Autonómicos
       ID: 1Lnp4jcdASi72QedN6HPviE2R6Gn8td2X
       Drive ID: 0AAZ0z1VatsQKUk9PVA
       Propietarios: 0

    Boletines Parlamentarios
       ID: 1tCPZdP81pM2sTuGMAFUYsvGNi45lnW8E
       Drive ID: 0AAZ0z1VatsQKUk9PVA
       Propietarios: 0


 Buscando 'Parlamentarios'...
 Encontradas 1 carpetas con 'Parlamentarios':
    Boletines Parlamentarios
       ID: 1tCPZdP81pM2sTuGMAFUYsvGNi45lnW8E
       Drive ID: 0AAZ0z1VatsQKUk9PVA
       Propietarios: 0


 Buscando 'Autonómicos'...
 Encontradas 1 carpetas con 'Au

##Proceso de iteración por la carpeta de <u>**boletines parlamentarios**</u>, para identificar y levantar solamente los archivos que hayan sido modificados o subidos después de cierta fecha. Luego, los inserta debajo de la hoja correspondiente de base de datos unificada

### **  En el desarrollo, también incluyo otras operaciones de limpieza y transformación (los guión bajo de las fórmulas Excel, la modificación de las fechas, adaptación del timezone a Madrid, etc.)

In [9]:
from googleapiclient.discovery import build

# Credenciales y parámetros
# sheet_name = 'Boletines_unificado'
# sheet_id_unificado = '1T8l8omO07iWkUyB2tuc6wGf_RJhsgifWUgTb6L2LPN8'
# CWD = '/content/drive/Shareddrives/(02 MAD) Iniciativas - AAPP - JP/Boletines Parlamentarios'
# cutoff_date = pytz.timezone('Europe/Madrid').localize(datetime(2025, 7, 18, 9, 0, 0))  # Fecha de corte -> año, mes, día, hora, minuto, segundo (IMPORTANTE)
# # scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/spreadsheets"]
# # creds = ServiceAccountCredentials.from_json_keyfile_name('/content/drive/Shareddrives/(02 MAD) Iniciativas - AAPP - JP/Automatización_py/test-automation-gsheets-5fa673850271.json', scope)
# # client = gspread.authorize(creds)

# Acceder a la hoja de cálculo unificada
sheet_unificado = client.open_by_key('1T8l8omO07iWkUyB2tuc6wGf_RJhsgifWUgTb6L2LPN8')
folder_id = '1tCPZdP81pM2sTuGMAFUYsvGNi45lnW8E'
def obtener_sheets_en_carpeta(folder_id, cutoff_date):
    """
    Obtiene todas las hojas de Google Sheets en una carpeta específica
    que han sido modificadas después de la fecha de corte
    """
    print(f" Buscando hojas en carpeta: {folder_id}")
    print(f" Fecha de corte: {cutoff_date}")

    # Consulta para obtener archivos de Google Sheets en la carpeta
    query = f"'{folder_id}' in parents and mimeType='application/vnd.google-apps.spreadsheet' and trashed=false"

    try:
        results = drive_service.files().list(
            q=query,
            fields="files(id, name, modifiedTime, createdTime)",
            pageSize=100,
            supportsAllDrives=True,  #  CLAVE: Para Shared Drives
            includeItemsFromAllDrives=True  #  CLAVE: Para Shared Drives
        ).execute()

        files = results.get('files', [])
        print(f" Total de Google Sheets encontrados: {len(files)}")

        if not files:
            print(" No se encontraron archivos de Google Sheets en la carpeta")
            return []

        sheets_modificadas = []

        for file in files:
            print(f"\n Procesando archivo: {file['name']}")

            # Convertir fecha de modificación a datetime
            modified_time_str = file['modifiedTime']
            modified_time = datetime.fromisoformat(modified_time_str.replace('Z', '+00:00'))
            modified_time = modified_time.astimezone(madrid_tz)

            print(f"    Fecha de modificación: {modified_time}")
            print(f"    Fecha de corte: {cutoff_date}")

            # Verificar si fue modificado después de la fecha de corte
            if modified_time > cutoff_date:
                print(f"    INCLUIDO: Modificado después de la fecha de corte")
                sheets_modificadas.append({
                    'id': file['id'],
                    'name': file['name'],
                    'modified_time': modified_time
                })
            else:
                print(f"    EXCLUIDO: No modificado después de la fecha de corte")

        print(f"\n Total de hojas que cumplen criterio: {len(sheets_modificadas)}")
        return sheets_modificadas

    except Exception as e:
        print(f" Error al obtener hojas de la carpeta: {e}")
        return []

def leer_datos_sheet(sheet_id):
    """
    Lee todos los datos de una hoja de Google Sheets y los convierte a DataFrame
    """
    print(f" Leyendo datos de la hoja: {sheet_id}")

    try:
        sheet = client.open_by_key(sheet_id)

        # Leer la primera hoja
        worksheet = sheet.get_worksheet(0)
        print(f"    Nombre de la hoja: {worksheet.title}")

        # Obtener todos los datos
        datos = worksheet.get_all_values()
        print(f"    Filas totales: {len(datos)}")

        if not datos or len(datos) < 2:
            print(f"    Hoja vacía o solo tiene encabezados")
            return pd.DataFrame()

        # Crear DataFrame
        encabezados = datos[0]
        filas = datos[1:]

        print(f"    Encabezados: {encabezados}")
        print(f"    Filas de datos: {len(filas)}")

        df = pd.DataFrame(filas, columns=encabezados)
        print(f"    DataFrame creado exitosamente: {df.shape}")

        return df

    except Exception as e:
        print(f"    Error al leer la hoja {sheet_id}: {e}")
        return pd.DataFrame()

def limpiar_dataframe_parlamentarios(dfp):
    """Limpia el DataFrame de boletines parlamentarios"""
    if dfp.empty:
        return []

    columns_to_replace = ['Sector', 'Marco geográfico', 'Proponente']
    for column in columns_to_replace:
        if column in dfp.columns:
            dfp[column] = dfp[column].astype(str).apply(
                lambda x: x.replace("Castilla_La_Mancha", "Castilla-La Mancha") if "Castilla_La_Mancha" in x else x.replace('_', ' ')
            )

    # Convertir las fechas a cadenas en formato DD/MM/YYYY
    if 'Fecha publicación' in dfp.columns:
        dfp['Fecha publicación'] = pd.to_datetime(dfp['Fecha publicación'], errors='coerce').dt.strftime('%d/%m/%Y')

    # Rellenar valores nulos y convertir a lista de listas
    dfp_cleaned = dfp.fillna('')
    data_cleaned = dfp_cleaned.values.tolist()
    return data_cleaned

def find_empty_row(worksheet):
    """Encuentra la primera fila verdaderamente vacía en la hoja"""
    all_values = worksheet.get_all_values()

    for index, row in enumerate(all_values, 1):
        if all(cell == '' for cell in row):
            return index

    return len(all_values) + 1

def obtener_sheet_id(sheet, worksheet_name):
    """Obtiene el sheetId de una hoja específica"""
    sheets_metadata = sheet.fetch_sheet_metadata()
    sheets = sheets_metadata['sheets']

    for s in sheets:
        if s['properties']['title'] == worksheet_name:
            return s['properties']['sheetId']

    raise Exception(f"No se encontró una hoja con el nombre: {worksheet_name}")

def aplicar_formato_fecha(sheet_id, worksheet_index, rango, columna_fecha):
    """Aplica el formato de fecha DD/MM/YYYY en la columna especificada"""
    service = build('sheets', 'v4', credentials=creds)

    requests = [
        {
            "repeatCell": {
                "range": {
                    "sheetId": worksheet_index,
                    "startRowIndex": rango[0],
                    "endRowIndex": rango[1],
                    "startColumnIndex": columna_fecha,
                    "endColumnIndex": columna_fecha + 1
                },
                "cell": {
                    "userEnteredFormat": {
                        "numberFormat": {
                            "type": "DATE",
                            "pattern": "DD/MM/YYYY"
                        }
                    }
                },
                "fields": "userEnteredFormat.numberFormat"
            }
        }
    ]

    body = {'requests': requests}
    service.spreadsheets().batchUpdate(spreadsheetId=sheet_id, body=body).execute()

# ============================================================================
# PROCESAMIENTO DE BOLETINES PARLAMENTARIOS
# ============================================================================

def procesar_boletines_parlamentarios():
    """Procesa boletines parlamentarios desde Google Sheets"""
    print("=== PROCESANDO BOLETINES PARLAMENTARIOS ===")

    folder_id = FOLDER_IDS['parlamentarios']
    cutoff_date = pytz.timezone('Europe/Madrid').localize(datetime(2025, 7, 30, 8, 25, 0))

    # Obtener hojas modificadas después de la fecha de corte
    sheets_modificadas = obtener_sheets_en_carpeta(folder_id, cutoff_date)

    if not sheets_modificadas:
        print("No se encontraron hojas modificadas después de la fecha de corte")
        return

    # ORDENAR LAS HOJAS POR FECHA DE MODIFICACIÓN (más antigua primero)
    sheets_modificadas.sort(key=lambda x: x['modified_time'])

    print(f"\nOrden de procesamiento:")
    for i, sheet_info in enumerate(sheets_modificadas, 1):
        print(f"  {i}. {sheet_info['name']} - {sheet_info['modified_time']}")

    dataframes = []

    for sheet_info in sheets_modificadas:
        print(f"\n Procesando hoja: {sheet_info['name']}")
        print(f"    Fecha de modificación: {sheet_info['modified_time']}")

        # Leer datos de la hoja
        df = leer_datos_sheet(sheet_info['id'])

        if not df.empty:
            dataframes.append(df)
            print(f"    DataFrame creado para: {sheet_info['name']}")

            # Limpiar el DataFrame
            data_cleaned = limpiar_dataframe_parlamentarios(df)

            if data_cleaned:
                print(f"    Insertando {len(data_cleaned)} filas en hoja unificada")

                # Obtener la hoja de destino (segunda hoja, índice 1)
                worksheet_destino = sheet_unificado.get_worksheet(1)
                sheet_id = obtener_sheet_id(sheet_unificado, worksheet_destino.title)

                # Obtener el índice de la fila donde insertar
                indice_fila_insercion = find_empty_row(worksheet_destino)
                print(f"    Insertando en fila: {indice_fila_insercion}")

                # Insertar los datos
                worksheet_destino.insert_rows(data_cleaned, indice_fila_insercion, value_input_option='USER_ENTERED')
                print("    Datos insertados correctamente")

                # Aplicar formato de fecha (columna J, índice 9)
                start_row = indice_fila_insercion - 1
                end_row = start_row + len(data_cleaned)
                aplicar_formato_fecha('1T8l8omO07iWkUyB2tuc6wGf_RJhsgifWUgTb6L2LPN8', sheet_id, (start_row, end_row), 9)
                print("    Formato de fecha aplicado")
            else:
                print("    No hay datos válidos para insertar")
        else:
            print("    DataFrame vacío, saltando hoja")

def main():
    """Función principal que ejecuta todo el proceso"""
    print(" INICIANDO PROCESAMIENTO DE GOOGLE SHEETS")
    print(f" Hora actual en Madrid: {datetime.now(madrid_tz)}")
    print("=" * 60)

    try:
        # 1. Procesar cada tipo de hoja
        procesar_boletines_parlamentarios()

        print("\n PROCESO COMPLETADO EXITOSAMENTE")

    except Exception as e:
        print(f"\n Error durante el procesamiento: {e}")
        import traceback
        traceback.print_exc()
        raise

if __name__ == "__main__":
    main()

 INICIANDO PROCESAMIENTO DE GOOGLE SHEETS
 Hora actual en Madrid: 2025-07-31 13:09:22.916225+02:00
=== PROCESANDO BOLETINES PARLAMENTARIOS ===
 Buscando hojas en carpeta: 1tCPZdP81pM2sTuGMAFUYsvGNi45lnW8E
 Fecha de corte: 2025-07-30 08:25:00+02:00
 Total de Google Sheets encontrados: 10

 Procesando archivo: 250731 Boletines Parlamentarios
    Fecha de modificación: 2025-07-31 10:24:44.462000+02:00
    Fecha de corte: 2025-07-30 08:25:00+02:00
    INCLUIDO: Modificado después de la fecha de corte

 Procesando archivo: 250730 Boletines Parlamentarios
    Fecha de modificación: 2025-07-30 10:03:43.938000+02:00
    Fecha de corte: 2025-07-30 08:25:00+02:00
    INCLUIDO: Modificado después de la fecha de corte

 Procesando archivo: 250729 Boletines Parlamentarios
    Fecha de modificación: 2025-07-29 12:28:08.312000+02:00
    Fecha de corte: 2025-07-30 08:25:00+02:00
    EXCLUIDO: No modificado después de la fecha de corte

 Procesando archivo: 250724 Boletines Parlamentarios
    Fecha de 

/tmp/ipython-input-3457615097.py:124: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  dfp['Fecha publicación'] = pd.to_datetime(dfp['Fecha publicación'], errors='coerce').dt.strftime('%d/%m/%Y')


    Insertando en fila: 18212
    Datos insertados correctamente
    Formato de fecha aplicado

 Procesando hoja: 250731 Boletines Parlamentarios
    Fecha de modificación: 2025-07-31 10:24:44.462000+02:00
 Leyendo datos de la hoja: 175vqr24ibAkKdxxuIkVhR7_lPje6fHAUIXbhcQCIMng
    Nombre de la hoja: Boletines parlamentarios
    Filas totales: 11
    Encabezados: ['ID', 'Sector', 'Sector2', 'Subsector', 'Tema', 'Legislatura', 'Marco geográfico', 'Título de la iniciativa', 'Link', 'Fecha publicación', 'Proponente', 'Tipo de iniciativa']
    Filas de datos: 10
    DataFrame creado exitosamente: (10, 12)
    DataFrame creado para: 250731 Boletines Parlamentarios
    Insertando 10 filas en hoja unificada


/tmp/ipython-input-3457615097.py:124: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  dfp['Fecha publicación'] = pd.to_datetime(dfp['Fecha publicación'], errors='coerce').dt.strftime('%d/%m/%Y')


    Insertando en fila: 18225
    Datos insertados correctamente
    Formato de fecha aplicado

 PROCESO COMPLETADO EXITOSAMENTE


## Crear Dataframe completo [histórico + > cutoff date] de boletines parlamentarios para luego exportarlos desde BigQuery

In [ ]:
# Credenciales
sheet_id_unificado = '1T8l8omO07iWkUyB2tuc6wGf_RJhsgifWUgTb6L2LPN8'

scope = ["https://spreadsheets.google.com/feeds",
         "https://www.googleapis.com/auth/drive",
         "https://www.googleapis.com/auth/spreadsheets"]

creds = ServiceAccountCredentials.from_json_keyfile_name(
    '/content/drive/Shareddrives/(02 MAD) Iniciativas - AAPP - JP/Automatización_py/test-automation-gsheets-5fa673850271.json',
    scope
)

client = gspread.authorize(creds)

# ============================================================================
# FUNCIONES DE CORRECCIÓN DE FECHAS - ALGUNOS BOLETINES ANTIGUOS TENÍAN ESTE PROBLEMA
# ============================================================================

def corregir_fechas_americanas(df, columna_fecha='Fecha publicación'):
    """
    Corrige fechas en formato americano (MM/DD/YYYY) a europeo (DD/MM/YYYY)
    """
    if columna_fecha not in df.columns:
        print(f" Columna '{columna_fecha}' no encontrada")
        return df

    print(f" Corrigiendo fechas americanas en columna '{columna_fecha}'...")

    # Mostrar ejemplos de fechas originales
    fechas_ejemplo = df[columna_fecha].head().tolist()
    print(f"    Ejemplos de fechas originales: {fechas_ejemplo}")

    def procesar_fecha(fecha_str):
        """Procesa una fecha individual"""
        if not fecha_str or fecha_str == '':
            return ''

        try:
            # Convertir a string si no lo es
            fecha_str = str(fecha_str).strip()

            # Intentar primero como formato europeo DD/MM/YYYY
            try:
                fecha_obj = pd.to_datetime(fecha_str, format='%d/%m/%Y', dayfirst=True)
                return fecha_obj.strftime('%d/%m/%Y')
            except:
                pass

            # Si falla, intentar como formato americano MM/DD/YYYY
            try:
                fecha_obj = pd.to_datetime(fecha_str, format='%m/%d/%Y', dayfirst=False)
                # Convertir a formato europeo DD/MM/YYYY
                return fecha_obj.strftime('%d/%m/%Y')
            except:
                pass

            # Si ambos fallan, intentar parseo automático
            try:
                fecha_obj = pd.to_datetime(fecha_str, dayfirst=True)
                return fecha_obj.strftime('%d/%m/%Y')
            except:
                print(f"    No se pudo procesar fecha: {fecha_str}")
                return fecha_str  # Devolver original si no se puede procesar

        except Exception as e:
            print(f"    Error procesando fecha '{fecha_str}': {e}")
            return fecha_str

    # Aplicar corrección a todas las fechas
    fechas_originales = df[columna_fecha].copy()
    df[columna_fecha] = df[columna_fecha].apply(procesar_fecha)

    # Mostrar ejemplos de fechas corregidas
    fechas_corregidas = df[columna_fecha].head().tolist()
    print(f"    Ejemplos de fechas corregidas: {fechas_corregidas}")

    # Contar cuántas se corrigieron
    cambios = (fechas_originales != df[columna_fecha]).sum()
    print(f"    Fechas corregidas: {cambios}")

    return df

# ============================================================================
# FUNCIÓN PRINCIPAL
# ============================================================================

def obtener_dataframe_historico():
    """
    Obtiene SOLO los datos históricos de la BBDD unificada
    (sin buscar archivos nuevos)
    """
    print("=== OBTENIENDO DATAFRAME HISTÓRICO DE BOLETINES PARLAMENTARIOS ===")

    try:
        # Abrir la hoja unificada
        sheet = client.open_by_key(sheet_id_unificado)
        worksheet = sheet.get_worksheet(1)  # Segunda hoja (parlamentarios)

        # Obtener todos los datos
        datos_historicos = worksheet.get_all_values()

        if not datos_historicos or len(datos_historicos) < 2:
            print(" No hay datos históricos disponibles")
            return pd.DataFrame()

        # Crear DataFrame
        encabezados = datos_historicos[0]
        filas = datos_historicos[1:]
        df_historico = pd.DataFrame(filas, columns=encabezados)

        print(f" Datos históricos encontrados: {len(df_historico)} filas")
        print(f" Columnas: {df_historico.columns.tolist()}")

        # Limpiar columnas vacías
        df_historico = df_historico.loc[:, df_historico.columns != '']
        df_historico = df_historico.dropna(axis=1, how='all')

        # Limpiar filas vacías
        filas_antes = len(df_historico)
        df_historico = df_historico.dropna(how='all')

        # Filtrar filas donde columnas principales están vacías
        columnas_principales = ['ID', 'Título de la iniciativa', 'Fecha publicación']
        columnas_existentes = [col for col in columnas_principales if col in df_historico.columns]

        if columnas_existentes:
            mask_vacias = (df_historico[columnas_existentes] == '').all(axis=1)
            df_historico = df_historico[~mask_vacias]

        filas_despues = len(df_historico)
        filas_eliminadas = filas_antes - filas_despues

        if filas_eliminadas > 0:
            print(f" Filas vacías eliminadas: {filas_eliminadas}")

        #  CORRECCIÓN DE FECHAS AMERICANAS
        if 'Fecha publicación' in df_historico.columns:
            df_historico = corregir_fechas_americanas(df_historico, 'Fecha publicación')

        # Limpiar texto
        columns_to_replace = ['Sector', 'Marco geográfico', 'Proponente']
        for column in columns_to_replace:
            if column in df_historico.columns:
                df_historico[column] = df_historico[column].astype(str).apply(
                    lambda x: x.replace("Castilla_La_Mancha", "Castilla-La Mancha") if "Castilla_La_Mancha" in x else x.replace('_', ' ')
                )

        # Rellenar nulos
        df_historico = df_historico.fillna('')

        print(f" DataFrame histórico limpio: {df_historico.shape}")

        return df_historico

    except Exception as e:
        print(f" Error al obtener datos históricos: {e}")
        return pd.DataFrame()

# ============================================================================
# EJECUCIÓN - MAIN
# ============================================================================

def main():
    """Función principal"""
    print(" OBTENIENDO SOLO DATOS HISTÓRICOS DE BOLETINES PARLAMENTARIOS")
    print(" INCLUYE CORRECCIÓN DE FECHAS AMERICANAS (MM/DD/YYYY → DD/MM/YYYY)")
    print("=" * 60)

    df_historico = obtener_dataframe_historico()

    if not df_historico.empty:
        print(f"\n DATAFRAME HISTÓRICO CREADO:")
        print(f"    Dimensiones: {df_historico.shape}")
        print(f"    Columnas: {len(df_historico.columns)}")
        print(f"    Filas: {len(df_historico)}")

        print("\n Columnas del DataFrame:")
        for i, col in enumerate(df_historico.columns, 1):
            print(f"   {i}. {col}")

        print("\n Últimas 5 filas:")
        print(df_historico.tail().to_string())

        # Verificar fechas si existe la columna
        if 'Fecha publicación' in df_historico.columns:
            fechas_no_vacias = df_historico['Fecha publicación'][df_historico['Fecha publicación'] != '']
            if not fechas_no_vacias.empty:
                print(f"\n Rango de fechas: {fechas_no_vacias.min()} - {fechas_no_vacias.max()}")

                # Verificar formato de fechas
                fechas_muestra = fechas_no_vacias.head(10).tolist()
                print(f" Muestra de fechas (formato DD/MM/YYYY): {fechas_muestra}")

        print("\n PROCESO COMPLETADO EXITOSAMENTE")
        return df_historico
    else:
        print("\n No se pudo crear el DataFrame histórico")
        return None

if __name__ == '__main__':
    dfp_historico = main()

    if dfp_historico is not None:
        print(f"\n DataFrame histórico disponible en la variable 'dfp_historico'")
        print(f" Forma: {dfp_historico.shape}")
        print("\n Este DataFrame contiene SOLO los datos históricos de la BBDD unificada")
        print(" Las fechas americanas (MM/DD/YYYY) han sido corregidas a formato europeo (DD/MM/YYYY)")
    else:
        print("\n No se pudo crear el DataFrame histórico")

# # Credenciales
# sheet_id_unificado = '1T8l8omO07iWkUyB2tuc6wGf_RJhsgifWUgTb6L2LPN8'

# scope = ["https://spreadsheets.google.com/feeds",
#          "https://www.googleapis.com/auth/drive",
#          "https://www.googleapis.com/auth/spreadsheets"]

# creds = ServiceAccountCredentials.from_json_keyfile_name(
#     '/content/drive/Shareddrives/(02 MAD) Iniciativas - AAPP - JP/Automatización_py/test-automation-gsheets-5fa673850271.json',
#     scope
# )

# client = gspread.authorize(creds)

# # ============================================================================
# # FUNCIONES DE CORRECCIÓN DE FECHAS MEJORADAS PARA LOOKER
# # ============================================================================

# def corregir_fechas_para_looker(df, columna_fecha='Fecha publicación'):
#     """
#     Corrige fechas y las convierte al formato YYYY-MM-DD compatible con Looker
#     MANTIENE COMO STRING para evitar errores de PyArrow, pero en formato ISO
#     """
#     if columna_fecha not in df.columns:
#         print(f" Columna '{columna_fecha}' no encontrada")
#         return df

#     print(f" 🔧 Corrigiendo fechas para Looker en columna '{columna_fecha}'...")

#     # Contadores para diagnóstico
#     fechas_procesadas = 0
#     fechas_americanas_corregidas = 0
#     fechas_europeas_mantenidas = 0
#     fechas_con_error = 0
#     fechas_vacias = 0

#     def procesar_fecha_para_looker(fecha_str):
#         """Procesa una fecha individual y la convierte a formato YYYY-MM-DD"""
#         nonlocal fechas_procesadas, fechas_americanas_corregidas, fechas_europeas_mantenidas, fechas_con_error, fechas_vacias

#         fechas_procesadas += 1

#         if not fecha_str or fecha_str == '' or str(fecha_str).strip() == '':
#             fechas_vacias += 1
#             return None  # Para compatibilidad con BigQuery

#         try:
#             fecha_str = str(fecha_str).strip()

#             # MÉTODO 1: Intentar formato europeo DD/MM/YYYY
#             try:
#                 fecha_obj = pd.to_datetime(fecha_str, format='%d/%m/%Y', dayfirst=True)
#                 fechas_europeas_mantenidas += 1
#                 return fecha_obj.strftime('%Y-%m-%d')  # Formato ISO para Looker
#             except:
#                 pass

#             # MÉTODO 2: Intentar formato americano MM/DD/YYYY
#             try:
#                 fecha_obj = pd.to_datetime(fecha_str, format='%m/%d/%Y', dayfirst=False)
#                 fechas_americanas_corregidas += 1
#                 return fecha_obj.strftime('%Y-%m-%d')  # Formato ISO para Looker
#             except:
#                 pass

#             # MÉTODO 3: Intentar otros formatos comunes
#             formatos_alternativos = [
#                 '%Y-%m-%d',  # Ya en formato ISO
#                 '%d-%m-%Y',  # DD-MM-YYYY
#                 '%m-%d-%Y',  # MM-DD-YYYY
#                 '%Y/%m/%d',  # YYYY/MM/DD
#             ]

#             for formato in formatos_alternativos:
#                 try:
#                     fecha_obj = pd.to_datetime(fecha_str, format=formato)
#                     fechas_europeas_mantenidas += 1
#                     return fecha_obj.strftime('%Y-%m-%d')
#                 except:
#                     continue

#             # MÉTODO 4: Parseo automático como último recurso
#             try:
#                 fecha_obj = pd.to_datetime(fecha_str, dayfirst=True)
#                 fechas_europeas_mantenidas += 1
#                 return fecha_obj.strftime('%Y-%m-%d')
#             except:
#                 pass

#             # Si nada funciona, registrar error
#             print(f"    ⚠️  No se pudo procesar fecha: '{fecha_str}'")
#             fechas_con_error += 1
#             return None

#         except Exception as e:
#             print(f"    ❌ Error procesando fecha '{fecha_str}': {e}")
#             fechas_con_error += 1
#             return None

#     # Aplicar corrección a todas las fechas
#     print(f"    📊 Procesando {df[columna_fecha].notna().sum()} fechas...")

#     df[columna_fecha] = df[columna_fecha].apply(procesar_fecha_para_looker)

#     # Mostrar estadísticas del procesamiento
#     print(f"    ✅ RESUMEN DEL PROCESAMIENTO:")
#     print(f"       Total procesadas: {fechas_procesadas}")
#     print(f"       Fechas vacías: {fechas_vacias}")
#     print(f"       Fechas europeas (DD/MM/YYYY): {fechas_europeas_mantenidas}")
#     print(f"       Fechas americanas corregidas (MM/DD/YYYY): {fechas_americanas_corregidas}")
#     print(f"       Fechas con error: {fechas_con_error}")

#     # Verificar rango de fechas válidas
#     fechas_validas = df[df[columna_fecha].notna()][columna_fecha]
#     if not fechas_validas.empty:
#         fecha_min = fechas_validas.min()
#         fecha_max = fechas_validas.max()
#         print(f"    📅 Rango de fechas: {fecha_min} → {fecha_max}")
#         print(f"    🎯 Formato final: YYYY-MM-DD (compatible con Looker)")

#         # Mostrar distribución por años para diagnóstico
#         fechas_por_año = pd.to_datetime(fechas_validas).dt.year.value_counts().sort_index()
#         print(f"    📊 Distribución por años:")
#         for año, cantidad in fechas_por_año.items():
#             print(f"       {año}: {cantidad} registros")

#     return df

# # ============================================================================
# # FUNCIÓN PRINCIPAL MEJORADA
# # ============================================================================

# def obtener_dataframe_historico():
#     """
#     Obtiene SOLO los datos históricos de la BBDD unificada
#     CON FECHAS EN FORMATO COMPATIBLE CON LOOKER
#     """
#     print("=== OBTENIENDO DATAFRAME HISTÓRICO CON FECHAS PARA LOOKER ===")

#     try:
#         # Abrir la hoja unificada
#         sheet = client.open_by_key(sheet_id_unificado)
#         worksheet = sheet.get_worksheet(1)  # Segunda hoja (parlamentarios)

#         # Obtener todos los datos
#         datos_historicos = worksheet.get_all_values()

#         if not datos_historicos or len(datos_historicos) < 2:
#             print(" No hay datos históricos disponibles")
#             return pd.DataFrame()

#         # Crear DataFrame
#         encabezados = datos_historicos[0]
#         filas = datos_historicos[1:]
#         df_historico = pd.DataFrame(filas, columns=encabezados)

#         print(f" 📋 Datos históricos encontrados: {len(df_historico)} filas")
#         print(f" 📋 Columnas: {df_historico.columns.tolist()}")

#         # Limpiar columnas vacías
#         df_historico = df_historico.loc[:, df_historico.columns != '']
#         df_historico = df_historico.dropna(axis=1, how='all')

#         # Limpiar filas vacías
#         filas_antes = len(df_historico)
#         df_historico = df_historico.dropna(how='all')

#         # Filtrar filas donde columnas principales están vacías
#         columnas_principales = ['ID', 'Título de la iniciativa', 'Fecha publicación']
#         columnas_existentes = [col for col in columnas_principales if col in df_historico.columns]

#         if columnas_existentes:
#             mask_vacias = (df_historico[columnas_existentes] == '').all(axis=1)
#             df_historico = df_historico[~mask_vacias]

#         filas_despues = len(df_historico)
#         filas_eliminadas = filas_antes - filas_despues

#         if filas_eliminadas > 0:
#             print(f" 🧹 Filas vacías eliminadas: {filas_eliminadas}")

#         # *** CORRECCIÓN DE FECHAS MEJORADA PARA LOOKER ***
#         if 'Fecha publicación' in df_historico.columns:
#             df_historico = corregir_fechas_para_looker(df_historico, 'Fecha publicación')
#         else:
#             print(" ⚠️  Columna 'Fecha publicación' no encontrada")

#         # Limpiar texto
#         columns_to_replace = ['Sector', 'Marco geográfico', 'Proponente']
#         for column in columns_to_replace:
#             if column in df_historico.columns:
#                 df_historico[column] = df_historico[column].astype(str).apply(
#                     lambda x: x.replace("Castilla_La_Mancha", "Castilla-La Mancha") if "Castilla_La_Mancha" in x else x.replace('_', ' ')
#                 )

#         # Rellenar nulos (pero NO en fecha para mantener None)
#         columnas_texto = [col for col in df_historico.columns if col != 'Fecha publicación']
#         df_historico[columnas_texto] = df_historico[columnas_texto].fillna('')

#         print(f" ✅ DataFrame histórico limpio: {df_historico.shape}")

#         return df_historico

#     except Exception as e:
#         print(f" ❌ Error al obtener datos históricos: {e}")
#         return pd.DataFrame()

# # ============================================================================
# # EJECUCIÓN - MAIN MEJORADA
# # ============================================================================

# def main():
#     """Función principal con diagnóstico mejorado"""
#     print(" 🔧 OBTENIENDO DATOS HISTÓRICOS CON FECHAS COMPATIBLES PARA LOOKER")
#     print(" 📅 CONVIERTE FECHAS A FORMATO YYYY-MM-DD (ISO)")
#     print(" 🎯 CORRIGE FECHAS AMERICANAS (MM/DD/YYYY) Y EUROPEAS (DD/MM/YYYY)")
#     print("=" * 80)

#     df_historico = obtener_dataframe_historico()

#     if not df_historico.empty:
#         print(f"\n ✅ DATAFRAME HISTÓRICO CREADO:")
#         print(f"    📊 Dimensiones: {df_historico.shape}")
#         print(f"    📋 Columnas: {len(df_historico.columns)}")
#         print(f"    📄 Filas: {len(df_historico)}")

#         # Diagnóstico específico de fechas
#         if 'Fecha publicación' in df_historico.columns:
#             fechas_col = df_historico['Fecha publicación']
#             fechas_validas = fechas_col[fechas_col.notna()]
#             fechas_nulas = fechas_col.isna().sum()

#             print(f"\n 📅 DIAGNÓSTICO DE FECHAS:")
#             print(f"    Fechas válidas: {len(fechas_validas)}")
#             print(f"    Fechas nulas: {fechas_nulas}")

#             if not fechas_validas.empty:
#                 print(f"    Rango temporal: {fechas_validas.min()} → {fechas_validas.max()}")
#                 print(f"    Formato: YYYY-MM-DD (compatible con Looker)")

#                 # Verificar que las fechas están en orden
#                 fechas_ordenadas = fechas_validas.sort_values()
#                 print(f"    Primeras 5 fechas: {fechas_ordenadas.head().tolist()}")
#                 print(f"    Últimas 5 fechas: {fechas_ordenadas.tail().tolist()}")

#                 # Diagnóstico por años
#                 try:
#                     años = pd.to_datetime(fechas_validas).dt.year.value_counts().sort_index()
#                     print(f"    📊 Datos por año:")
#                     for año, cantidad in años.items():
#                         print(f"       {año}: {cantidad} registros")

#                     if len(años) > 1:
#                         print(f"    ✅ Datos históricos detectados desde {años.index.min()} hasta {años.index.max()}")
#                     else:
#                         print(f"    ⚠️  Solo hay datos del año {años.index[0]} - puede que falten datos históricos")

#                 except Exception as e:
#                     print(f"    ⚠️  Error analizando años: {e}")

#         print(f"\n 🎉 PROCESO COMPLETADO EXITOSAMENTE")
#         print(f" 📊 Ahora el DataFrame debería ser compatible con Looker")
#         print(f" 🎯 Las fechas están en formato YYYY-MM-DD estándar")

#         return df_historico
#     else:
#         print("\n ❌ No se pudo crear el DataFrame histórico")
#         return None

# if __name__ == '__main__':
#     dfp_historico = main()

#     if dfp_historico is not None:
#         print(f"\n 📊 DataFrame histórico disponible en la variable 'dfp_historico'")
#         print(f" 📐 Forma: {dfp_historico.shape}")
#         print(f"\n 🎯 LISTO PARA LOOKER:")
#         print(f" ✅ Fechas en formato YYYY-MM-DD")
#         print(f" ✅ Compatible con análisis temporal")
#         print(f" ✅ Fechas americanas corregidas")
#     else:
#         print(f"\n ❌ No se pudo crear el DataFrame histórico")

 OBTENIENDO SOLO DATOS HISTÓRICOS DE BOLETINES PARLAMENTARIOS
 INCLUYE CORRECCIÓN DE FECHAS AMERICANAS (MM/DD/YYYY → DD/MM/YYYY)
=== OBTENIENDO DATAFRAME HISTÓRICO DE BOLETINES PARLAMENTARIOS ===
 Datos históricos encontrados: 18210 filas
 Columnas: ['ID', 'Sector', 'Sector2', 'Subsector', 'Tema', 'Legislatura', 'Marco geográfico', 'Título de la iniciativa', 'Link', 'Fecha publicación', 'Proponente', 'Tipo de iniciativa', '']
 Corrigiendo fechas americanas en columna 'Fecha publicación'...
    Ejemplos de fechas originales: ['3/05/2023', '3/05/2023', '3/05/2023', '2/05/2023', '2/05/2023']
    Ejemplos de fechas corregidas: ['03/05/2023', '03/05/2023', '03/05/2023', '02/05/2023', '02/05/2023']
    Fechas corregidas: 3498
 DataFrame histórico limpio: (18210, 12)

 DATAFRAME HISTÓRICO CREADO:
    Dimensiones: (18210, 12)
    Columnas: 12
    Filas: 18210

 Columnas del DataFrame:
   1. ID
   2. Sector
   3. Sector2
   4. Subsector
   5. Tema
   6. Legislatura
   7. Marco geográfico
   8. 

In [ ]:
dfp_historico

,ID,Sector,Sector2,Subsector,Tema,Legislatura,Marco geográfico,Título de la iniciativa,Link,Fecha publicación,Proponente,Tipo de iniciativa
0,1,Alimentación,,,,XIV,España,Proposición de Ley en apoyo del sistema alimen...,https://www.congreso.es/public_oficiales/L14/C...,2023-05-03,Congreso - Grupo Popular,Proposición de ley de Grupos Parlamentarios de...
1,2,Economía Circular,,,,XIV,España,Proposición no de Ley sobre impulso de la rena...,https://www.congreso.es/public_oficiales/L14/C...,2023-05-03,Congreso - Grupo Socialista,Proposición de ley de Grupos Parlamentarios de...
2,3,Salud,,Salud - Cáncer,,XIV,España,Moción ante el Pleno por la que se insta al Go...,https://www.senado.es/legis14/publicaciones/pd...,2023-05-03,Congreso - Pleno,Moción
3,4,Primario,,Primario - Cárnico,,XI,Andalucía,Solicitud de comparecencia de la consejera de ...,https://www.parlamentodeandalucia.es/webdinami...,2023-05-02,"Comisión de Agricultura, Pesca y Alimentación",Comparecencia del Gobierno en Comisión
4,5,Pesca,,,,XI,Andalucía,Proposición no de ley relativa a defensa de la...,https://www.parlamentodeandalucia.es/webdinami...,2023-05-02,Andalucía - Grupo Popular,Proposición no de Ley ante el Pleno
...,...,...,...,...,...,...,...,...,...,...,...,...
18205,18206,Energía,,,,XV,España,Energías renovables autóctonas y cuantificació...,https://www.congreso.es/entradap/l15p/e7/e_007...,2025-07-28,Congreso - Grupo VOX,Pregunta al Gobierno con respuesta escrita
18206,18207,Digitalización,,,,XV,España,Autorización para transferir al Gobierno Vasco...,https://www.congreso.es/entradap/l15p/e7/e_007...,2025-07-28,Congreso - Grupo VOX,Pregunta al Gobierno con respuesta escrita
18207,18208,Defensa,,,,XV,España,Medidas concretas que piensa adoptar el Gobier...,https://www.congreso.es/entradap/l15p/e7/e_007...,2025-07-28,Congreso - Grupo VOX,Pregunta al Gobierno con respuesta escrita
18208,18209,Energía,,,,XV,España,Prohibición de la publicidad de los productos ...,https://www.congreso.es/entradap/l15p/e7/e_007...,2025-07-28,Congreso - Grupo VOX,Pregunta al Gobierno con respuesta escrita


## Crear DataFrame con boletines parlamentarios posteriores a cutoff_date

In [10]:
# Credenciales
sheet_id_unificado = '1T8l8omO07iWkUyB2tuc6wGf_RJhsgifWUgTb6L2LPN8'
folder_id_parlamentarios = '1tCPZdP81pM2sTuGMAFUYsvGNi45lnW8E' #Puede estar o no, ya que hago la call más arriba

#  IMPORTANTE: Ajustar esta fecha para evitar duplicados
cutoff_date = pytz.timezone('Europe/Madrid').localize(datetime(2025, 7, 30, 9, 0, 0))

madrid_tz = pytz.timezone('Europe/Madrid')

scope = ["https://spreadsheets.google.com/feeds",
         "https://www.googleapis.com/auth/drive",
         "https://www.googleapis.com/auth/spreadsheets"]

creds = ServiceAccountCredentials.from_json_keyfile_name(
    '/content/drive/Shareddrives/(02 MAD) Iniciativas - AAPP - JP/Automatización_py/test-automation-gsheets-5fa673850271.json',
    scope
)

client = gspread.authorize(creds)
drive_service = build('drive', 'v3', credentials=creds)

# ============================================================================
# FUNCIONES DE CORRECCIÓN DE FECHAS - ANTIGUOS BOLETINES TENÍAN ESTE PROBLEA
# ============================================================================

def corregir_fechas_americanas(df, columna_fecha='Fecha publicación'):
    """
    Corrige fechas en formato americano (MM/DD/YYYY) a europeo (DD/MM/YYYY)
    """
    if columna_fecha not in df.columns:
        print(f" Columna '{columna_fecha}' no encontrada")
        return df

    print(f" Corrigiendo fechas americanas en columna '{columna_fecha}'...")

    # Mostrar ejemplos de fechas originales
    fechas_ejemplo = df[columna_fecha].head().tolist()
    print(f"    Ejemplos de fechas originales: {fechas_ejemplo}")

    def procesar_fecha(fecha_str):
        """Procesa una fecha individual"""
        if not fecha_str or fecha_str == '':
            return ''

        try:
            # Convertir a string si no lo es
            fecha_str = str(fecha_str).strip()

            # Intentar primero como formato europeo DD/MM/YYYY
            try:
                fecha_obj = pd.to_datetime(fecha_str, format='%d/%m/%Y', dayfirst=True)
                return fecha_obj.strftime('%d/%m/%Y')
            except:
                pass

            # Si falla, intentar como formato americano MM/DD/YYYY
            try:
                fecha_obj = pd.to_datetime(fecha_str, format='%m/%d/%Y', dayfirst=False)
                # Convertir a formato europeo DD/MM/YYYY
                return fecha_obj.strftime('%d/%m/%Y')
            except:
                pass

            # Si ambos fallan, intentar parseo automático
            try:
                fecha_obj = pd.to_datetime(fecha_str, dayfirst=True)
                return fecha_obj.strftime('%d/%m/%Y')
            except:
                print(f"    No se pudo procesar fecha: {fecha_str}")
                return fecha_str  # Devolver original si no se puede procesar

        except Exception as e:
            print(f"    Error procesando fecha '{fecha_str}': {e}")
            return fecha_str

    # Aplicar corrección a todas las fechas
    fechas_originales = df[columna_fecha].copy()
    df[columna_fecha] = df[columna_fecha].apply(procesar_fecha)

    # Mostrar ejemplos de fechas corregidas
    fechas_corregidas = df[columna_fecha].head().tolist()
    print(f"    Ejemplos de fechas corregidas: {fechas_corregidas}")

    # Contar cuántas se corrigieron
    cambios = (fechas_originales != df[columna_fecha]).sum()
    print(f"    Fechas corregidas: {cambios}")

    return df

# ============================================================================
# FUNCIONES PRINCIPALES
# ============================================================================

def obtener_ultima_fecha_historico():
    """
    Obtiene la fecha más reciente del histórico para ayudar a definir cutoff_date
    """
    try:
        sheet = client.open_by_key(sheet_id_unificado)
        worksheet = sheet.get_worksheet(1)  # Segunda hoja (parlamentarios)

        datos = worksheet.get_all_values()
        if not datos or len(datos) < 2:
            return None

        df = pd.DataFrame(datos[1:], columns=datos[0])

        if 'Fecha publicación' in df.columns:
            fechas_validas = df['Fecha publicación'][df['Fecha publicación'] != '']
            if not fechas_validas.empty:
                fecha_maxima = fechas_validas.max()
                print(f" Fecha más reciente en histórico: {fecha_maxima}")
                return fecha_maxima

        return None

    except Exception as e:
        print(f" Error al obtener última fecha: {e}")
        return None

def obtener_sheets_realmente_nuevos(folder_id, cutoff_date):
    """
    Obtiene SOLO las hojas que son realmente nuevas (modificadas después de cutoff_date)
    ORDENADAS POR FECHA DE MODIFICACIÓN (más antigua primero)
    """
    print(f" Buscando hojas REALMENTE nuevas después de {cutoff_date}")
    print(f"  IMPORTANTE: Solo procesará archivos modificados después de esta fecha")

    query = f"'{folder_id}' in parents and mimeType='application/vnd.google-apps.spreadsheet' and trashed=false"

    try:
        results = drive_service.files().list(
            q=query,
            fields="files(id, name, modifiedTime)",
            pageSize=100,
            supportsAllDrives=True,
            includeItemsFromAllDrives=True
        ).execute()

        files = results.get('files', [])
        print(f" Total de Google Sheets en carpeta: {len(files)}")

        sheets_nuevos = []

        for file in files:
            modified_time_str = file['modifiedTime']
            modified_time = datetime.fromisoformat(modified_time_str.replace('Z', '+00:00'))
            modified_time = modified_time.astimezone(madrid_tz)

            print(f"    {file['name']} - Modificado: {modified_time}")

            if modified_time > cutoff_date:  # Usar > en lugar de >= para ser más estricto
                sheets_nuevos.append({
                    'id': file['id'],
                    'name': file['name'],
                    'modified_time': modified_time
                })
                print(f"       INCLUIDO: Posterior a cutoff_date")
            else:
                print(f"       EXCLUIDO: Anterior o igual a cutoff_date")

        # ORDENAR POR FECHA DE MODIFICACIÓN (más antigua primero)
        sheets_nuevos.sort(key=lambda x: x['modified_time'])

        print(f"\n Hojas realmente nuevas encontradas: {len(sheets_nuevos)}")
        if sheets_nuevos:
            print(" Orden de procesamiento (por fecha de modificación):")
            for i, sheet_info in enumerate(sheets_nuevos, 1):
                print(f"   {i}. {sheet_info['name']} - {sheet_info['modified_time']}")

        return sheets_nuevos

    except Exception as e:
        print(f" Error al buscar hojas: {e}")
        return []

def leer_google_sheet(sheet_id):
    """Lee una hoja de Google Sheets"""
    try:
        sheet = client.open_by_key(sheet_id)
        worksheet = sheet.get_worksheet(0)

        datos = worksheet.get_all_values()

        if not datos or len(datos) < 2:
            return pd.DataFrame()

        df = pd.DataFrame(datos[1:], columns=datos[0])

        # Filtrar filas vacías
        df = df.dropna(how='all')

        # Filtrar filas donde columnas principales están vacías
        columnas_principales = ['ID', 'Título de la iniciativa']
        columnas_existentes = [col for col in columnas_principales if col in df.columns]

        if columnas_existentes:
            mask_vacias = (df[columnas_existentes] == '').all(axis=1)
            df = df[~mask_vacias]

        return df

    except Exception as e:
        print(f" Error al leer hoja {sheet_id}: {e}")
        return pd.DataFrame()

def verificar_duplicados_con_historico(df_nuevos):
    """
    Verifica si hay duplicados entre los datos nuevos y el histórico
    """
    if df_nuevos.empty or 'ID' not in df_nuevos.columns:
        return df_nuevos

    print(" Verificando duplicados con histórico...")

    try:
        # Obtener IDs del histórico
        sheet = client.open_by_key(sheet_id_unificado)
        worksheet = sheet.get_worksheet(1)  # Segunda hoja (parlamentarios)

        datos_historicos = worksheet.get_all_values()
        if not datos_historicos or len(datos_historicos) < 2:
            print("    No hay datos históricos para comparar")
            return df_nuevos

        df_historico = pd.DataFrame(datos_historicos[1:], columns=datos_historicos[0])

        if 'ID' in df_historico.columns:
            ids_historicos = set(df_historico['ID'].tolist())
            ids_nuevos = set(df_nuevos['ID'].tolist())

            ids_duplicados = ids_nuevos & ids_historicos

            if ids_duplicados:
                print(f"    Se encontraron {len(ids_duplicados)} IDs duplicados:")
                for id_dup in list(ids_duplicados)[:5]:  # Mostrar solo los primeros 5
                    print(f"      - ID: {id_dup}")
                if len(ids_duplicados) > 5:
                    print(f"      ... y {len(ids_duplicados) - 5} más")

                # Eliminar duplicados
                # df_sin_duplicados = df_nuevos[~df_nuevos['ID'].isin(ids_duplicados)]

                # print(f"    Filas eliminadas por duplicación: {len(df_nuevos) - len(df_sin_duplicados)}")
                # print(f"    Filas únicas restantes: {len(df_sin_duplicados)}")

                # return df_sin_duplicados
                # Mantener duplicados
                print(f"    MANTENIENDO TODOS LOS DATOS (incluyendo {len(ids_duplicados)} duplicados)")
                print(f"    Total de filas procesadas: {len(df_nuevos)}")

                return df_nuevos  # ← Devolver TODOS los datos
            else:
                print("    No se encontraron duplicados")
                return df_nuevos
        else:
            print("    No se encontró columna 'ID' en histórico")
            return df_nuevos

    except Exception as e:
        print(f"    Error al verificar duplicados: {e}")
        return df_nuevos

def obtener_solo_datos_nuevos():
    """
    Obtiene SOLO los datos verdaderamente nuevos (sin duplicados)
    PROCESADOS EN ORDEN CRONOLÓGICO
    """
    print("=== OBTENIENDO SOLO DATOS NUEVOS DE BOLETINES PARLAMENTARIOS ===")
    print(f" Fecha de corte: {cutoff_date}")
    print(f" Solo procesará archivos modificados DESPUÉS de esta fecha")
    print(" INCLUYE CORRECCIÓN DE FECHAS AMERICANAS (MM/DD/YYYY → DD/MM/YYYY)")
    print(" PROCESAMIENTO EN ORDEN CRONOLÓGICO (más antigua primero)")
    print("=" * 60)

    # Mostrar fecha más reciente del histórico para referencia
    obtener_ultima_fecha_historico()

    # Obtener hojas realmente nuevas (YA ORDENADAS POR FECHA)
    sheets_nuevos = obtener_sheets_realmente_nuevos(folder_id_parlamentarios, cutoff_date)

    if not sheets_nuevos:
        print("\n No se encontraron hojas realmente nuevas")
        print(" Esto significa que no hay datos nuevos después de la fecha de corte")
        return pd.DataFrame()

    # Leer cada hoja nueva EN ORDEN CRONOLÓGICO
    df_nuevos_lista = []

    for i, sheet_info in enumerate(sheets_nuevos, 1):
        print(f"\n [{i}/{len(sheets_nuevos)}] Procesando: {sheet_info['name']}")
        print(f"    Fecha de modificación: {sheet_info['modified_time']}")

        df_temp = leer_google_sheet(sheet_info['id'])

        if not df_temp.empty:
            df_nuevos_lista.append(df_temp)
            print(f"    DataFrame creado: {df_temp.shape} filas")
        else:
            print(f"    DataFrame vacío - sin datos válidos")

    if not df_nuevos_lista:
        print("\n No se pudieron crear DataFrames de las hojas nuevas")
        return pd.DataFrame()

    # Combinar DataFrames (ya están en orden cronológico)
    df_nuevos = pd.concat(df_nuevos_lista, ignore_index=True)
    print(f"\n DataFrame combinado (en orden cronológico): {df_nuevos.shape}")

    # Verificar duplicados con histórico
    df_sin_duplicados = verificar_duplicados_con_historico(df_nuevos)

    if df_sin_duplicados.empty:
        print("\n No quedan datos nuevos después de eliminar duplicados")
        return pd.DataFrame()

    # Filtrar por columnas comunes
    columnas_comunes = ['ID', 'Sector', 'Subsector', 'Tema', 'Legislatura', 'Marco geográfico',
                       'Título de la iniciativa', 'Link', 'Fecha publicación', 'Proponente', 'Tipo de iniciativa']

    columnas_existentes = [col for col in columnas_comunes if col in df_sin_duplicados.columns]
    columnas_faltantes = [col for col in columnas_comunes if col not in df_sin_duplicados.columns]

    if columnas_faltantes:
        print(f" Columnas faltantes: {columnas_faltantes}")

    print(f" Columnas seleccionadas: {columnas_existentes}")
    df_sin_duplicados = df_sin_duplicados[columnas_existentes]

    # 🔧 CORRECCIÓN DE FECHAS AMERICANAS
    if 'Fecha publicación' in df_sin_duplicados.columns:
        df_sin_duplicados = corregir_fechas_americanas(df_sin_duplicados, 'Fecha publicación')

    # Limpiar datos
    columns_to_replace = ['Sector', 'Marco geográfico', 'Proponente']
    for column in columns_to_replace:
        if column in df_sin_duplicados.columns:
            df_sin_duplicados[column] = df_sin_duplicados[column].astype(str).apply(
                lambda x: x.replace("Castilla_La_Mancha", "Castilla-La Mancha") if "Castilla_La_Mancha" in x else x.replace('_', ' ')
            )

    # Rellenar nulos
    df_sin_duplicados = df_sin_duplicados.fillna('')

    print(f"\n DataFrame final (solo nuevos, orden cronológico): {df_sin_duplicados.shape}")
    print(" Los datos se procesaron en orden: del 24/07 → 29/07")

    return df_sin_duplicados

# ============================================================================
# EJECUCIÓN - MAIN
# ============================================================================

def main():
    """Función principal"""
    print(" OBTENIENDO SOLO DATOS NUEVOS DE BOLETINES PARLAMENTARIOS")
    print("  IMPORTANTE: Este script evita duplicados con el histórico")
    print(" INCLUYE CORRECCIÓN DE FECHAS AMERICANAS (MM/DD/YYYY → DD/MM/YYYY)")
    print("=" * 60)

    df_nuevos = obtener_solo_datos_nuevos()

    if not df_nuevos.empty:
        print(f"\n DATOS NUEVOS OBTENIDOS:")
        print(f"    Dimensiones: {df_nuevos.shape}")
        print(f"    Columnas: {len(df_nuevos.columns)}")
        print(f"    Filas: {len(df_nuevos)}")

        print("\n Primeras 5 filas:")
        print(df_nuevos.head().to_string())

        # Verificar fechas
        if 'Fecha publicación' in df_nuevos.columns:
            fechas_no_vacias = df_nuevos['Fecha publicación'][df_nuevos['Fecha publicación'] != '']
            if not fechas_no_vacias.empty:
                print(f"\n Rango de fechas nuevas: {fechas_no_vacias.min()} - {fechas_no_vacias.max()}")

                # Verificar formato de fechas
                fechas_muestra = fechas_no_vacias.head(10).tolist()
                print(f" Muestra de fechas (formato DD/MM/YYYY): {fechas_muestra}")

                # Verificar si hay fechas futuras
                try:
                    fechas_temp = pd.to_datetime(fechas_no_vacias, format='%d/%m/%Y', errors='coerce')
                    fechas_futuras = fechas_temp > datetime.now()

                    if fechas_futuras.any():
                        print(f"    Fechas futuras encontradas: {fechas_futuras.sum()}")
                    else:
                        print(f"    No hay fechas futuras")
                except:
                    print("    No se pudo verificar fechas futuras")

        print("\n PROCESO COMPLETADO EXITOSAMENTE")
        print(" Estos datos son verdaderamente nuevos (sin duplicados)")
        print(" Las fechas americanas han sido corregidas a formato europeo")
        return df_nuevos
    else:
        print("\n No hay datos nuevos disponibles")
        print(" Posibles razones:")
        print("   - No hay archivos modificados después de la fecha de corte")
        print("   - Los archivos encontrados ya están en el histórico")
        print("   - Las hojas están vacías")
        print(f"\n Solución: Ajusta la fecha de corte en el código:")
        print(f"   cutoff_date = datetime(2025, 5, 30, 8, 0, 0)  # Más reciente")
        return None

if __name__ == '__main__':
    dfp_nuevos = main()

    if dfp_nuevos is not None:
        print(f"\n DataFrame de nuevos datos disponible en la variable 'dfp_nuevos'")
        print(f" Forma: {dfp_nuevos.shape}")
        print(" Las fechas americanas (MM/DD/YYYY) han sido corregidas a europeo (DD/MM/YYYY)")
    else:
        print("\n No se encontraron datos nuevos")

 OBTENIENDO SOLO DATOS NUEVOS DE BOLETINES PARLAMENTARIOS
  IMPORTANTE: Este script evita duplicados con el histórico
 INCLUYE CORRECCIÓN DE FECHAS AMERICANAS (MM/DD/YYYY → DD/MM/YYYY)
=== OBTENIENDO SOLO DATOS NUEVOS DE BOLETINES PARLAMENTARIOS ===
 Fecha de corte: 2025-07-30 09:00:00+02:00
 Solo procesará archivos modificados DESPUÉS de esta fecha
 INCLUYE CORRECCIÓN DE FECHAS AMERICANAS (MM/DD/YYYY → DD/MM/YYYY)
 PROCESAMIENTO EN ORDEN CRONOLÓGICO (más antigua primero)
 Fecha más reciente en histórico: 9/11/2023
 Buscando hojas REALMENTE nuevas después de 2025-07-30 09:00:00+02:00
  IMPORTANTE: Solo procesará archivos modificados después de esta fecha
 Total de Google Sheets en carpeta: 10
    250731 Boletines Parlamentarios - Modificado: 2025-07-31 10:24:44.462000+02:00
       INCLUIDO: Posterior a cutoff_date
    250730 Boletines Parlamentarios - Modificado: 2025-07-30 10:03:43.938000+02:00
       INCLUIDO: Posterior a cutoff_date
    250729 Boletines Parlamentarios - Modificado: 

/tmp/ipython-input-840940809.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[columna_fecha] = df[columna_fecha].apply(procesar_fecha)
/tmp/ipython-input-840940809.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sin_duplicados[column] = df_sin_duplicados[column].astype(str).apply(
/tmp/ipython-input-840940809.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [11]:
dfp_nuevos

,ID,Sector,Subsector,Tema,Legislatura,Marco geográfico,Título de la iniciativa,Link,Fecha publicación,Proponente,Tipo de iniciativa
0,18211,Primario,,,XV,España,Proposición no de Ley relativa a la creación d...,https://www.congreso.es/public_oficiales/L15/C...,30/07/2025,Congreso - Grupo Plural Sumar,Proposición no de Ley en Comisión
1,18212,Primario,,,XV,España,Proposición no de Ley relativa a la agricultur...,https://www.congreso.es/public_oficiales/L15/C...,30/07/2025,Congreso - Grupo Plural Sumar,Proposición no de Ley en Comisión
2,18213,Primario,,,XV,España,Proposición no de ley relativa a la regulación...,https://www.congreso.es/public_oficiales/L15/C...,30/07/2025,Congreso - Grupo Plural Sumar,Proposición no de Ley en Comisión
3,18214,Digitalización,,,XII,Andalucía,Pregunta relativa al Centro de Inteligencia Ar...,https://www.parlamentodeandalucia.es/webdinami...,30/07/2025,Andalucía - Grupo Mixto - Por Andalucía,Pregunta en Comisión
4,18215,Primario,,,XII,Andalucía,Pregunta relativa a las actuaciones previstas ...,https://www.parlamentodeandalucia.es/webdinami...,30/07/2025,Andalucía - Grupo VOX,Pregunta en Comisión
5,18216,Primario,,,XII,Andalucía,Pregunta relativa a los daños producidos en la...,https://www.parlamentodeandalucia.es/webdinami...,30/07/2025,Andalucía - Grupo VOX,Pregunta en Comisión
6,18217,Empresa,,,XII,Andalucía,Pregunta relativa a las startups,https://www.parlamentodeandalucia.es/webdinami...,30/07/2025,Andalucía - Grupo VOX,Pregunta en Comisión
7,18218,Turismo,,,XII,Andalucía,Pregunta relativa al compromiso andaluz de acc...,https://www.parlamentodeandalucia.es/webdinami...,30/07/2025,Andalucía - Grupo Popular,Pregunta en Comisión
8,18219,Turismo,,,XII,Andalucía,Pregunta relativa a la innovación turística (N...,https://www.parlamentodeandalucia.es/webdinami...,30/07/2025,Andalucía - Grupo Popular,Pregunta en Comisión
9,18220,Turismo,,,XII,Andalucía,Pregunta relativa al turismo MICE,https://www.parlamentodeandalucia.es/webdinami...,30/07/2025,Andalucía - Grupo Popular,Pregunta en Comisión


## Mismo proceso de iteración por la carpeta de <u>**boletines autonómicos**</u>, pero esta vez identifica y levanta solamente los archivos que hayan sido modificados o subidos después de cierta fecha. Luego, los inserta debajo de la hoja correspondiente de base de datos unificada


### *  En el desarrollo, también incluyo otras operaciones de limpieza y transformación (los guión bajo de las fórmulas Excel, la modificación de las fechas, adaptación del timezone a Madrid, etc.)

In [12]:
from googleapiclient.discovery import build

sheet_name = 'Boletines_unificado'
sheet_id_unificado = '1T8l8omO07iWkUyB2tuc6wGf_RJhsgifWUgTb6L2LPN8'
CWD = '/content/drive/Shareddrives/(02 MAD) Iniciativas - AAPP - JP/Boletines Autonómicos'
# cutoff_date = pytz.timezone('Europe/Madrid').localize(datetime(2025, 7, 15, 9, 0, 0))

# Credenciales de acceso a la API de Google Sheets
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name('/content/drive/Shareddrives/(02 MAD) Iniciativas - AAPP - JP/Automatización_py/test-automation-gsheets-5fa673850271.json', scope)
client = gspread.authorize(creds)


from googleapiclient.discovery import build

# Credenciales y parámetros
# sheet_name = 'Boletines_unificado'
# sheet_id_unificado = '1T8l8omO07iWkUyB2tuc6wGf_RJhsgifWUgTb6L2LPN8'
# CWD = '/content/drive/Shareddrives/(02 MAD) Iniciativas - AAPP - JP/Boletines Parlamentarios'
# cutoff_date = pytz.timezone('Europe/Madrid').localize(datetime(2025, 7, 18, 9, 0, 0))  # Fecha de corte -> año, mes, día, hora, minuto, segundo (IMPORTANTE)
# # scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/spreadsheets"]
# # creds = ServiceAccountCredentials.from_json_keyfile_name('/content/drive/Shareddrives/(02 MAD) Iniciativas - AAPP - JP/Automatización_py/test-automation-gsheets-5fa673850271.json', scope)
# # client = gspread.authorize(creds)

# Acceder a la hoja de cálculo unificada
sheet_unificado = client.open_by_key('1T8l8omO07iWkUyB2tuc6wGf_RJhsgifWUgTb6L2LPN8')
folder_id = '1Lnp4jcdASi72QedN6HPviE2R6Gn8td2X'
def obtener_sheets_en_carpeta(folder_id, cutoff_date):
    """
    Obtiene todas las hojas de Google Sheets en una carpeta específica
    que han sido modificadas después de la fecha de corte
    ORDENADAS POR FECHA DE MODIFICACIÓN (más antigua primero)
    """
    print(f" Buscando hojas en carpeta: {folder_id}")
    print(f" Fecha de corte: {cutoff_date}")

    # Consulta para obtener archivos de Google Sheets en la carpeta
    query = f"'{folder_id}' in parents and mimeType='application/vnd.google-apps.spreadsheet' and trashed=false"

    try:
        results = drive_service.files().list(
            q=query,
            fields="files(id, name, modifiedTime, createdTime)",
            pageSize=100,
            supportsAllDrives=True,  #  CLAVE: Para Shared Drives
            includeItemsFromAllDrives=True  #  CLAVE: Para Shared Drives
        ).execute()

        files = results.get('files', [])
        print(f" Total de Google Sheets encontrados: {len(files)}")

        if not files:
            print(" No se encontraron archivos de Google Sheets en la carpeta")
            return []

        sheets_modificadas = []

        for file in files:
            print(f"\n Procesando archivo: {file['name']}")

            # Convertir fecha de modificación a datetime
            modified_time_str = file['modifiedTime']
            modified_time = datetime.fromisoformat(modified_time_str.replace('Z', '+00:00'))
            modified_time = modified_time.astimezone(madrid_tz)

            print(f"    Fecha de modificación: {modified_time}")
            print(f"    Fecha de corte: {cutoff_date}")

            # Verificar si fue modificado después de la fecha de corte
            if modified_time > cutoff_date:
                print(f"    INCLUIDO: Modificado después de la fecha de corte")
                sheets_modificadas.append({
                    'id': file['id'],
                    'name': file['name'],
                    'modified_time': modified_time
                })
            else:
                print(f"    EXCLUIDO: No modificado después de la fecha de corte")

        # ORDENAR POR FECHA DE MODIFICACIÓN (más antigua primero)
        sheets_modificadas.sort(key=lambda x: x['modified_time'])

        print(f"\n Total de hojas que cumplen criterio: {len(sheets_modificadas)}")
        if sheets_modificadas:
            print(" Orden de procesamiento (por fecha de modificación):")
            for i, sheet_info in enumerate(sheets_modificadas, 1):
                print(f"   {i}. {sheet_info['name']} - {sheet_info['modified_time']}")

        return sheets_modificadas

    except Exception as e:
        print(f" Error al obtener hojas de la carpeta: {e}")
        return []

def leer_datos_sheet(sheet_id):
    """
    Lee todos los datos de una hoja de Google Sheets y los convierte a DataFrame
    """
    print(f" Leyendo datos de la hoja: {sheet_id}")

    try:
        sheet = client.open_by_key(sheet_id)

        # Leer la primera hoja
        worksheet = sheet.get_worksheet(0)
        print(f"    Nombre de la hoja: {worksheet.title}")

        # Obtener todos los datos
        datos = worksheet.get_all_values()
        print(f"    Filas totales: {len(datos)}")

        if not datos or len(datos) < 2:
            print(f"    Hoja vacía o solo tiene encabezados")
            return pd.DataFrame()

        # Crear DataFrame
        encabezados = datos[0]
        filas = datos[1:]

        print(f"    Encabezados: {encabezados}")
        print(f"    Filas de datos: {len(filas)}")

        df = pd.DataFrame(filas, columns=encabezados)
        print(f"    DataFrame creado exitosamente: {df.shape}")

        return df

    except Exception as e:
        print(f"    Error al leer la hoja {sheet_id}: {e}")
        return pd.DataFrame()

def limpiar_dataframe_autonomicos(dfa):
    """Limpia el DataFrame de boletines autonómicos"""
    if dfa.empty:
        return []

    columns_to_replace = ['Sector', 'Marco Geográfico', 'Proponente']
    for column in columns_to_replace:
        if column in dfa.columns:
            dfa[column] = dfa[column].astype(str).apply(
                lambda x: x.replace("Castilla_La_Mancha", "Castilla-La Mancha") if "Castilla_La_Mancha" in x else x.replace('_', ' ')
            )

    # Convertir las fechas a cadenas en formato DD/MM/YYYY
    if 'Fecha de publicación' in dfa.columns:
        dfa['Fecha de publicación'] = pd.to_datetime(dfa['Fecha de publicación'], errors='coerce').dt.strftime('%d/%m/%Y')

    # Rellenar valores nulos y convertir a lista de listas
    dfa_cleaned = dfa.fillna('')
    data_cleaned = dfa_cleaned.values.tolist()
    return data_cleaned

def find_empty_row(worksheet):
    """Encuentra la primera fila verdaderamente vacía en la hoja"""
    all_values = worksheet.get_all_values()

    for index, row in enumerate(all_values, 1):
        if all(cell == '' for cell in row):
            return index

    return len(all_values) + 1

def obtener_sheet_id(sheet, worksheet_name):
    """Obtiene el sheetId de una hoja específica"""
    sheets_metadata = sheet.fetch_sheet_metadata()
    sheets = sheets_metadata['sheets']

    for s in sheets:
        if s['properties']['title'] == worksheet_name:
            return s['properties']['sheetId']

    raise Exception(f"No se encontró una hoja con el nombre: {worksheet_name}")

def aplicar_formato_fecha(sheet_id, worksheet_index, rango, columna_fecha):
    """Aplica el formato de fecha DD/MM/YYYY en la columna especificada"""
    service = build('sheets', 'v4', credentials=creds)

    requests = [
        {
            "repeatCell": {
                "range": {
                    "sheetId": worksheet_index,
                    "startRowIndex": rango[0],
                    "endRowIndex": rango[1],
                    "startColumnIndex": columna_fecha,
                    "endColumnIndex": columna_fecha + 1
                },
                "cell": {
                    "userEnteredFormat": {
                        "numberFormat": {
                            "type": "DATE",
                            "pattern": "DD/MM/YYYY"
                        }
                    }
                },
                "fields": "userEnteredFormat.numberFormat"
            }
        }
    ]

    body = {'requests': requests}
    service.spreadsheets().batchUpdate(spreadsheetId=sheet_id, body=body).execute()

# ============================================================================
# PROCESAMIENTO DE BOLETINES AUTONÓMICOS
# ============================================================================

def procesar_boletines_autonomicos():
    """Procesa boletines autonómicos desde Google Sheets EN ORDEN CRONOLÓGICO"""
    print("\n=== PROCESANDO BOLETINES AUTONÓMICOS ===")

    folder_id = FOLDER_IDS['autonomicos']
    cutoff_date = pytz.timezone('Europe/Madrid').localize(datetime(2025, 7, 30, 9, 0, 0))

    # Obtener hojas modificadas después de la fecha de corte (YA ORDENADAS)
    sheets_modificadas = obtener_sheets_en_carpeta(folder_id, cutoff_date)

    if not sheets_modificadas:
        print("No se encontraron hojas modificadas después de la fecha de corte")
        return

    print(f"\nProcesando {len(sheets_modificadas)} hojas en orden cronológico...")

    dataframes = []

    for i, sheet_info in enumerate(sheets_modificadas, 1):
        print(f"\n[{i}/{len(sheets_modificadas)}] Procesando hoja: {sheet_info['name']}")
        print(f"    Fecha de modificación: {sheet_info['modified_time']}")

        # Leer datos de la hoja
        df = leer_datos_sheet(sheet_info['id'])

        if not df.empty:
            dataframes.append(df)
            print(f"    DataFrame creado para: {sheet_info['name']} ({df.shape[0]} filas)")

            # Limpiar el DataFrame
            data_cleaned = limpiar_dataframe_autonomicos(df)

            if data_cleaned:
                print(f"    Insertando {len(data_cleaned)} filas en hoja unificada")

                # Obtener la hoja de destino (primera hoja, índice 0)
                worksheet_destino = sheet_unificado.get_worksheet(0)
                sheet_id = obtener_sheet_id(sheet_unificado, worksheet_destino.title)

                # Obtener el índice de la fila donde insertar
                indice_fila_insercion = find_empty_row(worksheet_destino)
                print(f"    Insertando en fila: {indice_fila_insercion}")

                # Insertar los datos
                worksheet_destino.insert_rows(data_cleaned, indice_fila_insercion, value_input_option='USER_ENTERED')
                print("    Datos insertados correctamente")

                # Aplicar formato de fecha (columna H, índice 7)
                start_row = indice_fila_insercion - 1
                end_row = start_row + len(data_cleaned)
                aplicar_formato_fecha('1T8l8omO07iWkUyB2tuc6wGf_RJhsgifWUgTb6L2LPN8', sheet_id, (start_row, end_row), 7)
                print("    Formato de fecha aplicado")
            else:
                print("    No hay datos válidos para insertar después de la limpieza")
        else:
            print(f"    DataFrame vacío para: {sheet_info['name']}")

    print(f"\n✅ PROCESAMIENTO COMPLETADO EN ORDEN CRONOLÓGICO")
    print(f"   Total de hojas procesadas: {len(sheets_modificadas)}")
    print(f"   DataFrames creados: {len(dataframes)}")
    if sheets_modificadas:
        print(f"   Orden procesado: {sheets_modificadas[0]['modified_time'].strftime('%d/%m/%Y')} → {sheets_modificadas[-1]['modified_time'].strftime('%d/%m/%Y')}")

def main():
    """Función principal que ejecuta todo el proceso"""
    print(" INICIANDO PROCESAMIENTO DE GOOGLE SHEETS")
    print(f" Hora actual en Madrid: {datetime.now(madrid_tz)}")
    print("=" * 60)

    try:
        # 1. Procesar cada tipo de hoja
        procesar_boletines_autonomicos()

        print("\n PROCESO COMPLETADO EXITOSAMENTE")

    except Exception as e:
        print(f"\n Error durante el procesamiento: {e}")
        import traceback
        traceback.print_exc()
        raise

if __name__ == "__main__":
    main()

 INICIANDO PROCESAMIENTO DE GOOGLE SHEETS
 Hora actual en Madrid: 2025-07-31 13:11:10.365401+02:00

=== PROCESANDO BOLETINES AUTONÓMICOS ===
 Buscando hojas en carpeta: 1Lnp4jcdASi72QedN6HPviE2R6Gn8td2X
 Fecha de corte: 2025-07-30 09:00:00+02:00
 Total de Google Sheets encontrados: 10

 Procesando archivo: 250731 Boletines Autonómicos
    Fecha de modificación: 2025-07-31 12:40:36.256000+02:00
    Fecha de corte: 2025-07-30 09:00:00+02:00
    INCLUIDO: Modificado después de la fecha de corte

 Procesando archivo: 250730 Boletines Autonómicos
    Fecha de modificación: 2025-07-30 12:44:23.091000+02:00
    Fecha de corte: 2025-07-30 09:00:00+02:00
    INCLUIDO: Modificado después de la fecha de corte

 Procesando archivo: 250729 Boletines Autonómicos
    Fecha de modificación: 2025-07-29 12:39:26.391000+02:00
    Fecha de corte: 2025-07-30 09:00:00+02:00
    EXCLUIDO: No modificado después de la fecha de corte

 Procesando archivo: 250728 Boletines Autonómicos
    Fecha de modificación: 

/tmp/ipython-input-4187637516.py:146: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  dfa['Fecha de publicación'] = pd.to_datetime(dfa['Fecha de publicación'], errors='coerce').dt.strftime('%d/%m/%Y')


    Insertando en fila: 10993
    Datos insertados correctamente
    Formato de fecha aplicado

[2/2] Procesando hoja: 250731 Boletines Autonómicos
    Fecha de modificación: 2025-07-31 12:40:36.256000+02:00
 Leyendo datos de la hoja: 10dNy8xZVB4EhiIksOkDeSGjrlYd_Yzp5wgfmq66gD1s
    Nombre de la hoja: Boletines autonómicos
    Filas totales: 34
    Encabezados: ['ID', 'Sector', 'Sector2', 'Sector3', 'Subsector', 'Tema', 'Marco Geográfico', 'Fecha de publicación', 'Tipo de publicación', 'Tipo de iniciativa', 'Título iniciativa', 'Link', 'Proponente', 'Subgrupo']
    Filas de datos: 33
    DataFrame creado exitosamente: (33, 14)
    DataFrame creado para: 250731 Boletines Autonómicos (33 filas)
    Insertando 33 filas en hoja unificada


/tmp/ipython-input-4187637516.py:146: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  dfa['Fecha de publicación'] = pd.to_datetime(dfa['Fecha de publicación'], errors='coerce').dt.strftime('%d/%m/%Y')


    Insertando en fila: 11026
    Datos insertados correctamente
    Formato de fecha aplicado

✅ PROCESAMIENTO COMPLETADO EN ORDEN CRONOLÓGICO
   Total de hojas procesadas: 2
   DataFrames creados: 2
   Orden procesado: 30/07/2025 → 31/07/2025

 PROCESO COMPLETADO EXITOSAMENTE


## Creo Dataframe de boletines autonómicos para exportarlos luego a BigQuery

In [ ]:
# Credenciales
sheet_id_unificado = '1T8l8omO07iWkUyB2tuc6wGf_RJhsgifWUgTb6L2LPN8'

scope = ["https://spreadsheets.google.com/feeds",
         "https://www.googleapis.com/auth/drive",
         "https://www.googleapis.com/auth/spreadsheets"]

creds = ServiceAccountCredentials.from_json_keyfile_name(
    '/content/drive/Shareddrives/(02 MAD) Iniciativas - AAPP - JP/Automatización_py/test-automation-gsheets-5fa673850271.json',
    scope
)

client = gspread.authorize(creds)

# ============================================================================
# FUNCIÓN PRINCIPAL
# ============================================================================

def obtener_dataframe_historico():
    """
    Obtiene SOLO los datos históricos de la BBDD unificada
    (sin buscar archivos nuevos)
    """
    print("=== OBTENIENDO DATAFRAME HISTÓRICO DE BOLETINES AUTONÓMICOS ===")

    try:
        # Abrir la hoja unificada
        sheet = client.open_by_key(sheet_id_unificado)
        worksheet = sheet.get_worksheet(0)  # Primera hoja (autonómicos)

        # Obtener todos los datos
        datos_historicos = worksheet.get_all_values()

        if not datos_historicos or len(datos_historicos) < 2:
            print(" No hay datos históricos disponibles")
            return pd.DataFrame()

        # Crear DataFrame
        encabezados = datos_historicos[0]
        filas = datos_historicos[1:]
        df_historico = pd.DataFrame(filas, columns=encabezados)

        print(f" Datos históricos encontrados: {len(df_historico)} filas")
        print(f" Columnas: {df_historico.columns.tolist()}")

        # Limpiar columnas vacías
        df_historico = df_historico.loc[:, df_historico.columns != '']
        df_historico = df_historico.dropna(axis=1, how='all')

        # Limpiar filas vacías
        filas_antes = len(df_historico)
        df_historico = df_historico.dropna(how='all')

        # Filtrar filas donde columnas principales están vacías
        columnas_principales = ['ID', 'Título iniciativa', 'Fecha de publicación']
        columnas_existentes = [col for col in columnas_principales if col in df_historico.columns]

        if columnas_existentes:
            mask_vacias = (df_historico[columnas_existentes] == '').all(axis=1)
            df_historico = df_historico[~mask_vacias]

        filas_despues = len(df_historico)
        filas_eliminadas = filas_antes - filas_despues

        if filas_eliminadas > 0:
            print(f" Filas vacías eliminadas: {filas_eliminadas}")

        # Limpiar texto
        columns_to_replace = ['Sector', 'Marco Geográfico', 'Proponente']
        for column in columns_to_replace:
            if column in df_historico.columns:
                df_historico[column] = df_historico[column].astype(str).apply(
                    lambda x: x.replace("Castilla_La_Mancha", "Castilla-La Mancha") if "Castilla_La_Mancha" in x else x.replace('_', ' ')
                )

        # Rellenar nulos
        df_historico = df_historico.fillna('')

        print(f" DataFrame histórico limpio: {df_historico.shape}")

        return df_historico

    except Exception as e:
        print(f" Error al obtener datos históricos: {e}")
        return pd.DataFrame()

# ============================================================================
# EJECUCIÓN
# ============================================================================

def main():
    """Función principal"""
    print(" OBTENIENDO SOLO DATOS HISTÓRICOS DE BOLETINES AUTONÓMICOS")
    print("=" * 60)

    df_historico = obtener_dataframe_historico()

    if not df_historico.empty:
        print(f"\n DATAFRAME HISTÓRICO CREADO:")
        print(f"    Dimensiones: {df_historico.shape}")
        print(f"    Columnas: {len(df_historico.columns)}")
        print(f"    Filas: {len(df_historico)}")

        print("\n Columnas del DataFrame:")
        for i, col in enumerate(df_historico.columns, 1):
            print(f"   {i}. {col}")

        print("\n Últimas 5 filas:")
        print(df_historico.tail().to_string())

        # Verificar fechas si existe la columna
        if 'Fecha de publicación' in df_historico.columns:
            fechas_no_vacias = df_historico['Fecha de publicación'][df_historico['Fecha de publicación'] != '']
            if not fechas_no_vacias.empty:
                print(f"\n Rango de fechas: {fechas_no_vacias.min()} - {fechas_no_vacias.max()}")

        print("\n PROCESO COMPLETADO EXITOSAMENTE")
        return df_historico
    else:
        print("\n No se pudo crear el DataFrame histórico")
        return None

if __name__ == '__main__':
    dfa_historico = main()

    if dfa_historico is not None:
        print(f"\n DataFrame histórico disponible en la variable 'dfa_historico'")
        print(f" Forma: {dfa_historico.shape}")
        print("\n Este DataFrame contiene SOLO los datos históricos de la BBDD unificada")
        print(" Para obtener datos nuevos, usa el script separado de 'solo nuevos'")
    else:
        print("\n No se pudo crear el DataFrame histórico")

 OBTENIENDO SOLO DATOS HISTÓRICOS DE BOLETINES AUTONÓMICOS
=== OBTENIENDO DATAFRAME HISTÓRICO DE BOLETINES AUTONÓMICOS ===
 Datos históricos encontrados: 10844 filas
 Columnas: ['ID', 'Sector', 'Sector2', 'Sector3', 'Subsector', 'Tema', 'Marco Geográfico', 'Fecha de publicación', 'Tipo de publicación', 'Tipo de iniciativa', 'Título iniciativa', 'Link', 'Proponente', 'Subgrupo', '', '']
 DataFrame histórico limpio: (10844, 14)

 DATAFRAME HISTÓRICO CREADO:
    Dimensiones: (10844, 14)
    Columnas: 14
    Filas: 10844

 Columnas del DataFrame:
   1. ID
   2. Sector
   3. Sector2
   4. Sector3
   5. Subsector
   6. Tema
   7. Marco Geográfico
   8. Fecha de publicación
   9. Tipo de publicación
   10. Tipo de iniciativa
   11. Título iniciativa
   12. Link
   13. Proponente
   14. Subgrupo

 Últimas 5 filas:
          ID        Sector   Sector2 Sector3 Subsector Tema Marco Geográfico Fecha de publicación                Tipo de publicación Tipo de iniciativa                               

In [ ]:
from datetime import datetime

# Convertir de nuevo a datetime si hace falta (por si ya está en string)
dfa_historico['Fecha de publicación'] = pd.to_datetime(dfa_historico['Fecha de publicación'], dayfirst=True, errors='coerce')

# Definir la fecha actual
fecha_actual = datetime.now()

# Crear una máscara booleana
fechas_futuras_mask = dfa_historico['Fecha de publicación'] > fecha_actual

# Contar cuántas hay
cantidad_fechas_futuras = fechas_futuras_mask.sum()
print(f"Se encontraron {cantidad_fechas_futuras} fechas futuras.")


Se encontraron 0 fechas futuras.


In [ ]:
# Verificar el tipo de dato de la columna 'Fecha de publicación'
print(dfa_historico['Fecha de publicación'].dtype)

# Ver las primeras filas para verificar si las fechas están bien
print(dfa_historico['Fecha de publicación'].head())


datetime64[ns]
0   2023-05-05
1   2023-05-08
2   2023-05-08
3   2023-05-08
4   2023-05-08
Name: Fecha de publicación, dtype: datetime64[ns]


In [ ]:
# # Si la columna es de tipo 'object' (string), intentar convertirla a datetime
# if dfa_historico['Fecha de publicación'].dtype == 'object':
#     dfa_historico['Fecha de publicación'] = pd.to_datetime(dfa_historico['Fecha de publicación'], errors='coerce')
#     print("La columna ha sido convertida a tipo datetime.")
# else:
#     print("La columna ya es de tipo datetime.")


## Creo DataFrame de Boletines autonómicos pero solamente de los posteriores a cutoff_date

In [13]:
import os
import pandas as pd
from datetime import datetime
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
import pytz

# --- Parámetros y credenciales ---
sheet_name = 'Boletines_unificado'
sheet_id_unificado = '1T8l8omO07iWkUyB2tuc6wGf_RJhsgifWUgTb6L2LPN8'

# ID de la carpeta de Boletines Autonómicos en Google Drive
folder_id_autonomicos = '1Lnp4jcdASi72QedN6HPviE2R6Gn8td2X'

# Fecha de corte
cutoff_date = datetime(2025, 7, 30, 9, 0, 0)

# Zona horaria
madrid_tz = pytz.timezone('Europe/Madrid')

# Configuración de credenciales
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
    "https://www.googleapis.com/auth/spreadsheets"
]

creds = ServiceAccountCredentials.from_json_keyfile_name(
    '/content/drive/Shareddrives/(02 MAD) Iniciativas - AAPP - JP/Automatización_py/test-automation-gsheets-5fa673850271.json',
    scope
)

# Inicializar servicios
client = gspread.authorize(creds)
drive_service = build('drive', 'v3', credentials=creds)

# --- Funciones ---

def obtener_sheets_modificados_recientes(folder_id, cutoff_date):
    """
    Obtiene los Google Sheets modificados después de la fecha de corte
    ORDENADOS POR FECHA DE MODIFICACIÓN (más antigua primero)
    (equivalente a os.listdir() + verificación de fecha de modificación)
    """
    print(f" Buscando Google Sheets modificados después de {cutoff_date}")

    query = f"'{folder_id}' in parents and mimeType='application/vnd.google-apps.spreadsheet' and trashed=false"

    try:
        results = drive_service.files().list(
            q=query,
            fields="files(id, name, modifiedTime)",
            pageSize=100,
            supportsAllDrives=True,
            includeItemsFromAllDrives=True
        ).execute()

        files = results.get('files', [])
        print(f" Total de Google Sheets encontrados: {len(files)}")

        sheets_recientes = []

        for file in files:
            # Convertir fecha de modificación (sin zona horaria para comparar con cutoff_date)
            modified_time_str = file['modifiedTime']
            modified_time = datetime.fromisoformat(modified_time_str.replace('Z', '+00:00'))

            # Convertir a datetime sin timezone para comparar con cutoff_date
            modified_time_local = modified_time.replace(tzinfo=None)

            if modified_time_local >= cutoff_date:
                sheets_recientes.append({
                    'id': file['id'],
                    'name': file['name'],
                    'modified_time': modified_time_local
                })
                print(f"    {file['name']} - Modificado: {modified_time_local}")
            else:
                print(f"    {file['name']} - Modificado: {modified_time_local} (anterior a cutoff)")

        # ORDENAR POR FECHA DE MODIFICACIÓN (más antigua primero)
        sheets_recientes.sort(key=lambda x: x['modified_time'])

        print(f" Hojas que cumplen criterio: {len(sheets_recientes)}")
        if sheets_recientes:
            print(" Orden de procesamiento (por fecha de modificación):")
            for i, sheet_info in enumerate(sheets_recientes, 1):
                print(f"   {i}. {sheet_info['name']} - {sheet_info['modified_time']}")

        return sheets_recientes

    except Exception as e:
        print(f" Error al buscar hojas: {e}")
        return []

def leer_google_sheet(sheet_id):
    """
    Lee una hoja de Google Sheets y la convierte a DataFrame
    (equivalente a pd.read_excel())
    """
    try:
        sheet = client.open_by_key(sheet_id)
        worksheet = sheet.get_worksheet(0)  # Primera hoja

        # Obtener todos los datos
        datos = worksheet.get_all_values()

        if not datos or len(datos) < 2:
            print(f"    Hoja vacía o solo encabezados")
            return pd.DataFrame()

        # Crear DataFrame
        encabezados = datos[0]
        filas = datos[1:]
        df = pd.DataFrame(filas, columns=encabezados)

        # Filtrar filas completamente vacías
        df = df.dropna(how='all')

        # Filtrar filas donde las columnas principales están vacías
        columnas_principales = ['ID', 'Título iniciativa']
        columnas_existentes = [col for col in columnas_principales if col in df.columns]

        if columnas_existentes:
            mask_vacias = (df[columnas_existentes] == '').all(axis=1)
            df = df[~mask_vacias]

        return df

    except Exception as e:
        print(f"    Error al leer hoja {sheet_id}: {e}")
        return pd.DataFrame()

def limpiar_dataframe(dfp):
    """Limpia campos de texto y corrige nombres."""
    if dfp.empty:
        return dfp

    columns_to_replace = ['Sector', 'Marco Geográfico', 'Proponente']
    for column in columns_to_replace:
        if column in dfp.columns:
            dfp[column] = dfp[column].astype(str).apply(
                lambda x: x.replace("Castilla_La_Mancha", "Castilla-La Mancha") if "Castilla_La_Mancha" in x else x.replace('_', ' ')
            )
    return dfp

def obtener_nuevos_datos():
    """
    Devuelve un DataFrame con las hojas de Google Sheets modificadas a partir de la cutoff_date.
    PROCESADAS EN ORDEN CRONOLÓGICO
    """
    print("=== OBTENIENDO NUEVOS DATOS DE BOLETINES AUTONÓMICOS ===")
    print(" PROCESAMIENTO EN ORDEN CRONOLÓGICO (más antigua primero)")

    # Obtener hojas modificadas recientemente (YA ORDENADAS por fecha)
    sheets_recientes = obtener_sheets_modificados_recientes(folder_id_autonomicos, cutoff_date)

    if not sheets_recientes:
        print(" No se encontraron hojas modificadas después de la fecha de corte")
        return pd.DataFrame()

    print(f"\nProcesando {len(sheets_recientes)} hojas en orden cronológico...")

    # Leer cada hoja y crear DataFrames EN ORDEN CRONOLÓGICO
    df_nuevos = []

    for i, sheet_info in enumerate(sheets_recientes, 1):
        print(f"\n[{i}/{len(sheets_recientes)}] Procesando: {sheet_info['name']}")
        print(f"    Fecha de modificación: {sheet_info['modified_time']}")

        # Leer Google Sheet (equivalente a pd.read_excel())
        df_temp = leer_google_sheet(sheet_info['id'])

        if not df_temp.empty:
            df_nuevos.append(df_temp)
            print(f"    DataFrame creado: {df_temp.shape} filas")
        else:
            print(f"    DataFrame vacío - sin datos válidos")

    if not df_nuevos:
        print(" No se pudieron crear DataFrames de las hojas encontradas")
        return pd.DataFrame()

    # Combinar todos los DataFrames (ya están en orden cronológico)
    print(f"\n Combinando {len(df_nuevos)} DataFrames en orden cronológico...")
    dfa_nuevos = pd.concat(df_nuevos, ignore_index=True)

    print(f" DataFrame combinado inicial: {dfa_nuevos.shape}")

    # Filtrar por columnas comunes específicas de autonómicos
    columnas_comunes = ['ID', 'Sector', 'Sector2', 'Sector3', 'Subsector', 'Tema',
                       'Marco Geográfico', 'Fecha de publicación', 'Título iniciativa',
                       'Link', 'Proponente', 'Subgrupo']

    # Verificar qué columnas existen
    columnas_existentes = [col for col in columnas_comunes if col in dfa_nuevos.columns]
    columnas_faltantes = [col for col in columnas_comunes if col not in dfa_nuevos.columns]

    if columnas_faltantes:
        print(f" Columnas faltantes: {columnas_faltantes}")

    print(f" Columnas seleccionadas: {columnas_existentes}")
    dfa_nuevos = dfa_nuevos[columnas_existentes]

    # Convertir fecha de publicación, forzando el formato adecuado
    if 'Fecha de publicación' in dfa_nuevos.columns:
        print(" Procesando fechas...")

        # Convertir a datetime con formato europeo
        dfa_nuevos['Fecha de publicación'] = pd.to_datetime(
            dfa_nuevos['Fecha de publicación'],
            errors='coerce',
            dayfirst=True
        )

        # Verificar fechas inválidas
        fechas_invalidas = dfa_nuevos[dfa_nuevos['Fecha de publicación'].isna()]
        if not fechas_invalidas.empty:
            print(f" Fechas no válidas encontradas: {len(fechas_invalidas)}")

        # Reformatear para mantener el formato DD/MM/YYYY
        dfa_nuevos['Fecha de publicación'] = dfa_nuevos['Fecha de publicación'].dt.strftime('%d/%m/%Y')

    # Limpiar texto
    dfa_nuevos = limpiar_dataframe(dfa_nuevos)

    # Rellenar valores nulos
    dfa_nuevos = dfa_nuevos.fillna('')

    print(f" DataFrame final (orden cronológico): {dfa_nuevos.shape}")
    if sheets_recientes:
        print(f" Orden procesado: {sheets_recientes[0]['modified_time'].strftime('%d/%m/%Y')} → {sheets_recientes[-1]['modified_time'].strftime('%d/%m/%Y')}")

    return dfa_nuevos

# --- Ejecutar ---
print(" INICIANDO OBTENCIÓN DE NUEVOS DATOS AUTONÓMICOS DESDE GOOGLE SHEETS")
print(" PROCESAMIENTO EN ORDEN CRONOLÓGICO (más antigua → más reciente)")
print(f" Fecha de corte: {cutoff_date}")
print(f" Carpeta: {folder_id_autonomicos}")
print("=" * 60)

dfa_nuevos = obtener_nuevos_datos()

# --- Mostrar resultados iniciales ---
if not dfa_nuevos.empty:
    print(f"\n✅ ÉXITO: DataFrame creado con {len(dfa_nuevos)} nuevas filas")
    print("   Los datos se procesaron en orden cronológico correcto")
    print(" Columnas:", dfa_nuevos.columns.tolist())
    print("\n Muestra del DataFrame:")
    print(dfa_nuevos.head())

    # Mostrar información adicional
    print(f"\n Información adicional:")
    print(f"    Dimensiones: {dfa_nuevos.shape}")

    # Verificar rango de fechas si existe la columna
    if 'Fecha de publicación' in dfa_nuevos.columns:
        fechas_no_vacias = dfa_nuevos['Fecha de publicación'][dfa_nuevos['Fecha de publicación'] != '']
        if not fechas_no_vacias.empty:
            print(f"    Rango de fechas: {fechas_no_vacias.min()} - {fechas_no_vacias.max()}")

        # Verificar si hay fechas futuras
        fecha_actual = datetime.now()
        mask_futuras = pd.to_datetime(dfa_nuevos['Fecha de publicación'], format='%d/%m/%Y', errors='coerce') > fecha_actual

        if mask_futuras.any():
            print(f"    Se encontraron {mask_futuras.sum()} fechas futuras en los nuevos datos:")
            print(dfa_nuevos.loc[mask_futuras, ['ID', 'Fecha de publicación']].head())
        else:
            print("    No se encontraron fechas futuras en los nuevos datos.")

else:
    print("\n No hay nuevos datos posteriores a la fecha de corte.")
    print(" Posibles causas:")
    print("   - No hay hojas modificadas después de la fecha de corte")
    print("   - Las hojas están vacías")
    print("   - Problemas de permisos o conectividad")

print("\n PROCESO COMPLETADO - DATOS PROCESADOS EN ORDEN CRONOLÓGICO")

 INICIANDO OBTENCIÓN DE NUEVOS DATOS AUTONÓMICOS DESDE GOOGLE SHEETS
 PROCESAMIENTO EN ORDEN CRONOLÓGICO (más antigua → más reciente)
 Fecha de corte: 2025-07-30 09:00:00
 Carpeta: 1Lnp4jcdASi72QedN6HPviE2R6Gn8td2X
=== OBTENIENDO NUEVOS DATOS DE BOLETINES AUTONÓMICOS ===
 PROCESAMIENTO EN ORDEN CRONOLÓGICO (más antigua primero)
 Buscando Google Sheets modificados después de 2025-07-30 09:00:00
 Total de Google Sheets encontrados: 10
    250731 Boletines Autonómicos - Modificado: 2025-07-31 10:40:36.256000
    250730 Boletines Autonómicos - Modificado: 2025-07-30 10:44:23.091000
    250729 Boletines Autonómicos - Modificado: 2025-07-29 10:39:26.391000 (anterior a cutoff)
    250728 Boletines Autonómicos - Modificado: 2025-07-28 10:55:23.829000 (anterior a cutoff)
    250724 Boletines Autonómicos - Modificado: 2025-07-24 10:41:13.876000 (anterior a cutoff)
    250723 Boletines Autonómicos - Modificado: 2025-07-23 10:12:04.364000 (anterior a cutoff)
    250722 Boletines Autonómicos - Modi

In [14]:
dfa_nuevos.tail()

,ID,Sector,Sector2,Sector3,Subsector,Tema,Marco Geográfico,Fecha de publicación,Título iniciativa,Link,Proponente,Subgrupo
61,11053,Alimentación,Comercio,,,,Europa,31/07/2025,Reglamento de Ejecución (UE) 2025/1530 de la C...,https://eur-lex.europa.eu/legal-content/ES/TXT...,Unión Europea,Unión Europea
62,11054,Primario,,,,,Europa,31/07/2025,Reglamento de Ejecución (UE) 2025/1542 de la C...,https://eur-lex.europa.eu/legal-content/ES/TXT...,Unión Europea,Unión Europea
63,11055,Farmacéutico,,,Farmacéutico - Fitosanitarios,,Europa,31/07/2025,Reglamento de Ejecución (UE) 2025/1545 de la C...,https://eur-lex.europa.eu/legal-content/ES/TXT...,Unión Europea,Unión Europea
64,11056,Alimentación,,,,,Europa,31/07/2025,Reglamento de Ejecución (UE) 2025/1549 de la C...,https://eur-lex.europa.eu/legal-content/ES/TXT...,Unión Europea,Unión Europea
65,11057,Alimentación,Salud,,,,Europa,31/07/2025,Reglamento de Ejecución (UE) 2025/1560 de la C...,https://eur-lex.europa.eu/legal-content/ES/TXT...,Unión Europea,Unión Europea


## Mismo proceso de iteración por las carpeta de <u>**agendas parlamentarias**</u>, pero esta vez identifica y levanta solamente los archivos que hayan sido modificados o subidos después de cierta fecha. Luego, los inserta debajo de la hoja correspondiente de base de datos unificada

### ***  En el desarrollo, también incluyo otras operaciones de limpieza y transformación (los guión bajo de las fórmulas Excel, la modificación de las fechas, adaptación del timezone a Madrid, etc.)

In [ ]:
from datetime import datetime
from googleapiclient.discovery import build

sheet_name = 'Boletines_unificado'
sheet_id_unificado = '1T8l8omO07iWkUyB2tuc6wGf_RJhsgifWUgTb6L2LPN8'
CWD = '/content/drive/Shareddrives/(02 MAD) Iniciativas - AAPP - JP/Agendas parlamentarias'
# cutoff_date = madrid_tz.localize(datetime(2025, 7, 17, 12, 0, 0))  # Fecha de corte -> año, mes, día, hora, minuto y segundo

# Credenciales de acceso a la API de Google Sheets
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name('/content/drive/Shareddrives/(02 MAD) Iniciativas - AAPP - JP/Automatización_py/test-automation-gsheets-5fa673850271.json', scope)
client = gspread.authorize(creds)
gc = gspread.authorize(creds)

# Credenciales de acceso a la API de Google Sheets
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
# creds = ServiceAccountCredentials.from_json_keyfile_name('/content/drive/Shareddrives/(02 MAD) Iniciativas - AAPP - JP/Automatización_py/test-automation-gsheets-5fa673850271.json', scope)
client = gspread.authorize(creds)

# Credenciales y parámetros
# sheet_name = 'Boletines_unificado'
# sheet_id_unificado = '1T8l8omO07iWkUyB2tuc6wGf_RJhsgifWUgTb6L2LPN8'
# CWD = '/content/drive/Shareddrives/(02 MAD) Iniciativas - AAPP - JP/Boletines Parlamentarios'
# cutoff_date = pytz.timezone('Europe/Madrid').localize(datetime(2025, 7, 15, 9, 0, 0))  # Fecha de corte -> año, mes, día, hora, minuto, segundo (IMPORTANTE)
# # scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/spreadsheets"]
# # creds = ServiceAccountCredentials.from_json_keyfile_name('/content/drive/Shareddrives/(02 MAD) Iniciativas - AAPP - JP/Automatización_py/test-automation-gsheets-5fa673850271.json', scope)
# # client = gspread.authorize(creds)

# Acceder a la hoja de cálculo unificada
sheet_unificado = client.open_by_key('1T8l8omO07iWkUyB2tuc6wGf_RJhsgifWUgTb6L2LPN8')
folder_id = '1CCdTfcadG4MadY5Tp665yzhEBAzPQ1Km'

def obtener_sheets_en_carpeta(folder_id, cutoff_date):
    """
    Obtiene todas las hojas de Google Sheets en una carpeta específica
    que han sido modificadas después de la fecha de corte
    ORDENADAS POR FECHA DE MODIFICACIÓN (más antigua primero)
    """
    print(f" Buscando hojas en carpeta: {folder_id}")
    print(f" Fecha de corte: {cutoff_date}")

    # Consulta para obtener archivos de Google Sheets en la carpeta
    query = f"'{folder_id}' in parents and mimeType='application/vnd.google-apps.spreadsheet' and trashed=false"

    try:
        results = drive_service.files().list(
            q=query,
            fields="files(id, name, modifiedTime, createdTime)",
            pageSize=100,
            supportsAllDrives=True,  #  CLAVE: Para Shared Drives
            includeItemsFromAllDrives=True  #  CLAVE: Para Shared Drives
        ).execute()

        files = results.get('files', [])
        print(f" Total de Google Sheets encontrados: {len(files)}")

        if not files:
            print(" No se encontraron archivos de Google Sheets en la carpeta")
            return []

        sheets_modificadas = []

        for file in files:
            print(f"\n Procesando archivo: {file['name']}")

            # Convertir fecha de modificación a datetime
            modified_time_str = file['modifiedTime']
            modified_time = datetime.fromisoformat(modified_time_str.replace('Z', '+00:00'))
            modified_time = modified_time.astimezone(madrid_tz)

            print(f"    Fecha de modificación: {modified_time}")
            print(f"    Fecha de corte: {cutoff_date}")

            # Verificar si fue modificado después de la fecha de corte
            if modified_time > cutoff_date:
                print(f"   INCLUIDO: Modificado después de la fecha de corte")
                sheets_modificadas.append({
                    'id': file['id'],
                    'name': file['name'],
                    'modified_time': modified_time
                })
            else:
                print(f"    EXCLUIDO: No modificado después de la fecha de corte")

        # ORDENAR POR FECHA DE MODIFICACIÓN (más antigua primero)
        sheets_modificadas.sort(key=lambda x: x['modified_time'])

        print(f"\n Total de hojas que cumplen criterio: {len(sheets_modificadas)}")
        if sheets_modificadas:
            print(" Orden de procesamiento (por fecha de modificación):")
            for i, sheet_info in enumerate(sheets_modificadas, 1):
                print(f"   {i}. {sheet_info['name']} - {sheet_info['modified_time']}")

        return sheets_modificadas

    except Exception as e:
        print(f" Error al obtener hojas de la carpeta: {e}")
        return []

def leer_datos_sheet(sheet_id):
    """
    Lee todos los datos de una hoja de Google Sheets y los convierte a DataFrame
    """
    print(f" Leyendo datos de la hoja: {sheet_id}")

    try:
        sheet = client.open_by_key(sheet_id)

        # Leer la primera hoja
        worksheet = sheet.get_worksheet(0)
        print(f"    Nombre de la hoja: {worksheet.title}")

        # Obtener todos los datos
        datos = worksheet.get_all_values()
        print(f"    Filas totales: {len(datos)}")

        if not datos or len(datos) < 2:
            print(f"    Hoja vacía o solo tiene encabezados")
            return pd.DataFrame()

        # Crear DataFrame
        encabezados = datos[0]
        filas = datos[1:]

        print(f"    Encabezados: {encabezados}")
        print(f"    Filas de datos: {len(filas)}")

        df = pd.DataFrame(filas, columns=encabezados)
        print(f"    DataFrame creado exitosamente: {df.shape}")

        return df

    except Exception as e:
        print(f"    Error al leer la hoja {sheet_id}: {e}")
        return pd.DataFrame()

def limpiar_dataframe_agendas(dfag):
    """Limpia el DataFrame de agendas parlamentarias"""
    if dfag.empty:
        return []

    columns_to_replace = ['Sector', 'Marco geográfico', 'Proponente']
    for column in columns_to_replace:
        if column in dfag.columns:
            dfag[column] = dfag[column].astype(str).apply(
                lambda x: x.replace("Castilla_La_Mancha", "Castilla-La Mancha") if "Castilla_La_Mancha" in x else x.replace('_', ' ')
            )

    # Convertir las fechas a cadenas en formato DD/MM/YYYY
    if 'Fecha convocada' in dfag.columns:
        dfag['Fecha convocada'] = pd.to_datetime(dfag['Fecha convocada'], errors='coerce').dt.strftime('%d/%m/%Y')

    # Rellenar valores nulos y convertir a lista de listas
    dfag_cleaned = dfag.fillna('')
    data_cleaned = dfag_cleaned.values.tolist()
    return data_cleaned

def find_empty_row(worksheet):
    """Encuentra la primera fila verdaderamente vacía en la hoja"""
    all_values = worksheet.get_all_values()

    for index, row in enumerate(all_values, 1):
        if all(cell == '' for cell in row):
            return index

    return len(all_values) + 1

def obtener_sheet_id(sheet, worksheet_name):
    """Obtiene el sheetId de una hoja específica"""
    sheets_metadata = sheet.fetch_sheet_metadata()
    sheets = sheets_metadata['sheets']

    for s in sheets:
        if s['properties']['title'] == worksheet_name:
            return s['properties']['sheetId']

    raise Exception(f"No se encontró una hoja con el nombre: {worksheet_name}")

def aplicar_formato_fecha(sheet_id, worksheet_index, rango, columna_fecha):
    """Aplica el formato de fecha DD/MM/YYYY en la columna especificada"""
    service = build('sheets', 'v4', credentials=creds)

    requests = [
        {
            "repeatCell": {
                "range": {
                    "sheetId": worksheet_index,
                    "startRowIndex": rango[0],
                    "endRowIndex": rango[1],
                    "startColumnIndex": columna_fecha,
                    "endColumnIndex": columna_fecha + 1
                },
                "cell": {
                    "userEnteredFormat": {
                        "numberFormat": {
                            "type": "DATE",
                            "pattern": "DD/MM/YYYY"
                        }
                    }
                },
                "fields": "userEnteredFormat.numberFormat"
            }
        }
    ]

    body = {'requests': requests}
    service.spreadsheets().batchUpdate(spreadsheetId=sheet_id, body=body).execute()
# ============================================================================
# PROCESAMIENTO DE AGENGAS PARLAMENTARIAS
# ============================================================================

def procesar_agendas_parlamentarias():
    """Procesa agendas parlamentarias desde Google Sheets EN ORDEN CRONOLÓGICO"""
    print("\n=== PROCESANDO AGENDAS PARLAMENTARIAS ===")

    folder_id = FOLDER_IDS['agendas']
    cutoff_date = madrid_tz.localize(datetime(2025, 7, 27, 8, 0, 0))

    # Obtener hojas modificadas después de la fecha de corte (YA ORDENADAS)
    sheets_modificadas = obtener_sheets_en_carpeta(folder_id, cutoff_date)

    if not sheets_modificadas:
        print("No se encontraron hojas modificadas después de la fecha de corte")
        return

    print(f"\nProcesando {len(sheets_modificadas)} hojas en orden cronológico...")

    dataframes = []

    for i, sheet_info in enumerate(sheets_modificadas, 1):
        print(f"\n[{i}/{len(sheets_modificadas)}] Procesando hoja: {sheet_info['name']}")
        print(f"    Fecha de modificación: {sheet_info['modified_time']}")

        # Leer datos de la hoja
        df = leer_datos_sheet(sheet_info['id'])

        if not df.empty:
            dataframes.append(df)
            print(f"    DataFrame creado para: {sheet_info['name']} ({df.shape[0]} filas)")

            # Limpiar el DataFrame
            data_cleaned = limpiar_dataframe_agendas(df)

            if data_cleaned:
                print(f"    Insertando {len(data_cleaned)} filas en hoja unificada")

                # Obtener la hoja de destino (tercera hoja, índice 2)
                worksheet_destino = sheet_unificado.get_worksheet(2)
                sheet_id = obtener_sheet_id(sheet_unificado, worksheet_destino.title)

                # Obtener el índice de la fila donde insertar
                indice_fila_insercion = find_empty_row(worksheet_destino)
                print(f"    Insertando en fila: {indice_fila_insercion}")

                # Insertar los datos
                worksheet_destino.insert_rows(data_cleaned, indice_fila_insercion, value_input_option='USER_ENTERED')
                print("    Datos insertados correctamente")

                # Aplicar formato de fecha (columna H, índice 7)
                start_row = indice_fila_insercion - 1
                end_row = start_row + len(data_cleaned)
                aplicar_formato_fecha('1T8l8omO07iWkUyB2tuc6wGf_RJhsgifWUgTb6L2LPN8', sheet_id, (start_row, end_row), 7)
                print("    Formato de fecha aplicado")
            else:
                print("    No hay datos válidos para insertar después de la limpieza")
        else:
            print(f"    DataFrame vacío para: {sheet_info['name']}")

    print(f"\n PROCESAMIENTO DE AGENDAS COMPLETADO EN ORDEN CRONOLÓGICO")
    print(f"   Total de hojas procesadas: {len(sheets_modificadas)}")
    print(f"   DataFrames creados: {len(dataframes)}")
    if sheets_modificadas:
        print(f"   Orden procesado: {sheets_modificadas[0]['modified_time'].strftime('%d/%m/%Y')} → {sheets_modificadas[-1]['modified_time'].strftime('%d/%m/%Y')}")

def main():
    """Función principal que ejecuta todo el proceso"""
    print(" INICIANDO PROCESAMIENTO DE AGENDAS PARLAMENTARIAS")
    print(" PROCESAMIENTO EN ORDEN CRONOLÓGICO (más antigua → más reciente)")
    print(f" Hora actual en Madrid: {datetime.now(madrid_tz)}")
    print("=" * 60)

    try:
        # Procesar agendas parlamentarias en orden cronológico
        procesar_agendas_parlamentarias()

        print("\n PROCESO COMPLETADO EXITOSAMENTE")
        print("   Las agendas se procesaron en orden cronológico correcto")

    except Exception as e:
        print(f"\n Error durante el procesamiento: {e}")
        import traceback
        traceback.print_exc()
        raise

if __name__ == "__main__":
    main()

 INICIANDO PROCESAMIENTO DE AGENDAS PARLAMENTARIAS
 PROCESAMIENTO EN ORDEN CRONOLÓGICO (más antigua → más reciente)
 Hora actual en Madrid: 2025-07-29 11:54:50.178621+02:00

=== PROCESANDO AGENDAS PARLAMENTARIAS ===
 Buscando hojas en carpeta: 1CCdTfcadG4MadY5Tp665yzhEBAzPQ1Km
 Fecha de corte: 2025-07-27 08:00:00+02:00
 Total de Google Sheets encontrados: 3

 Procesando archivo: 250728 Agendas Parlamentarias
    Fecha de modificación: 2025-07-28 09:39:17.601000+02:00
    Fecha de corte: 2025-07-27 08:00:00+02:00
   INCLUIDO: Modificado después de la fecha de corte

 Procesando archivo: 250721 Agendas Parlamentarias
    Fecha de modificación: 2025-07-21 10:25:53.874000+02:00
    Fecha de corte: 2025-07-27 08:00:00+02:00
    EXCLUIDO: No modificado después de la fecha de corte

 Procesando archivo: 250714 Agendas Parlamentarias
    Fecha de modificación: 2025-07-16 17:22:39.715000+02:00
    Fecha de corte: 2025-07-27 08:00:00+02:00
    EXCLUIDO: No modificado después de la fecha de corte

/tmp/ipython-input-22-426769411.py:151: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  dfag['Fecha convocada'] = pd.to_datetime(dfag['Fecha convocada'], errors='coerce').dt.strftime('%d/%m/%Y')


    Insertando en fila: 4114
    Datos insertados correctamente
    Formato de fecha aplicado

✅ PROCESAMIENTO DE AGENDAS COMPLETADO EN ORDEN CRONOLÓGICO
   Total de hojas procesadas: 1
   DataFrames creados: 1
   Orden procesado: 28/07/2025 → 28/07/2025

✅ PROCESO COMPLETADO EXITOSAMENTE
   Las agendas se procesaron en orden cronológico correcto


## Ahora transformo agendas parlamentarias a Dataframe para exportar luego a BigQuery

In [ ]:
# ============================================================================
# CONFIGURACIÓN
# ============================================================================

# Credenciales
sheet_id_unificado = '1T8l8omO07iWkUyB2tuc6wGf_RJhsgifWUgTb6L2LPN8'

scope = ["https://spreadsheets.google.com/feeds",
         "https://www.googleapis.com/auth/drive",
         "https://www.googleapis.com/auth/spreadsheets"]

creds = ServiceAccountCredentials.from_json_keyfile_name(
    '/content/drive/Shareddrives/(02 MAD) Iniciativas - AAPP - JP/Automatización_py/test-automation-gsheets-5fa673850271.json',
    scope
)

client = gspread.authorize(creds)

# ============================================================================
# FUNCIONES DE CORRECCIÓN DE FECHAS
# ============================================================================

def corregir_fechas_agendas(df, columna_fecha='Fecha convocada'):
    """
    Corrige fechas en la columna de agendas parlamentarias
    Versión mejorada que maneja errores de Excel/Google Sheets
    """
    if columna_fecha not in df.columns:
        print(f" Columna '{columna_fecha}' no encontrada")
        return df

    print(f" Procesando fechas en columna '{columna_fecha}'...")

    # Mostrar ejemplos de fechas originales
    fechas_ejemplo = df[columna_fecha].head(10).tolist()
    print(f"    Ejemplos de fechas originales: {fechas_ejemplo}")

    # Contadores para estadísticas
    errores_value = 0
    fechas_corregidas = 0
    fechas_no_procesadas = 0

    def procesar_fecha(fecha_str):
        """Procesa una fecha individual con mejor manejo de errores"""
        nonlocal errores_value, fechas_corregidas, fechas_no_procesadas

        if not fecha_str or fecha_str == '':
            return ''

        # Convertir a string y limpiar
        fecha_str = str(fecha_str).strip()

        # Manejar errores específicos de Excel/Google Sheets
        if fecha_str in ['#VALUE!', '#ERROR!', '#REF!', '#N/A', '#DIV/0!']:
            errores_value += 1
            print(f"    Error de fórmula encontrado: {fecha_str}")
            return ''  # Devolver vacío en lugar del error

        try:
            # Intentar formato europeo DD/MM/YYYY
            try:
                fecha_obj = pd.to_datetime(fecha_str, format='%d/%m/%Y', dayfirst=True)
                fechas_corregidas += 1
                return fecha_obj.strftime('%d/%m/%Y')
            except:
                pass

            # Intentar formato con día sin cero inicial D/MM/YYYY
            try:
                fecha_obj = pd.to_datetime(fecha_str, format='%d/%m/%Y', dayfirst=True)
                fechas_corregidas += 1
                return fecha_obj.strftime('%d/%m/%Y')
            except:
                pass

            # Intentar formato americano MM/DD/YYYY
            try:
                fecha_obj = pd.to_datetime(fecha_str, format='%m/%d/%Y', dayfirst=False)
                fechas_corregidas += 1
                return fecha_obj.strftime('%d/%m/%Y')
            except:
                pass

            # Intentar formato con guiones DD-MM-YYYY
            try:
                fecha_obj = pd.to_datetime(fecha_str, format='%d-%m-%Y', dayfirst=True)
                fechas_corregidas += 1
                return fecha_obj.strftime('%d/%m/%Y')
            except:
                pass

            # Intentar parseo automático con dayfirst=True
            try:
                fecha_obj = pd.to_datetime(fecha_str, dayfirst=True)
                fechas_corregidas += 1
                return fecha_obj.strftime('%d/%m/%Y')
            except:
                pass

            # Si nada de lo anterior funciona, mantener el formato original
            fechas_no_procesadas += 1
            print(f"    No se pudo procesar fecha: {fecha_str}")
            return fecha_str

        except Exception as e:
            fechas_no_procesadas += 1
            print(f"    Error procesando fecha '{fecha_str}': {e}")
            return fecha_str

    # Aplicar corrección
    fechas_originales = df[columna_fecha].copy()
    df[columna_fecha] = df[columna_fecha].apply(procesar_fecha)

    # Mostrar resultados
    fechas_ejemplo_corregidas = df[columna_fecha].head(10).tolist()
    print(f"    Ejemplos de fechas corregidas: {fechas_ejemplo_corregidas}")

    # Estadísticas detalladas
    print(f"    Estadísticas de procesamiento:")
    print(f"       Fechas corregidas correctamente: {fechas_corregidas}")
    print(f"       Errores de fórmula (#VALUE!, etc.): {errores_value}")
    print(f"       Fechas no procesadas: {fechas_no_procesadas}")

    return df

def obtener_estadisticas_fechas(df, columna_fecha='Fecha convocada'):
    """
    Obtiene estadísticas detalladas sobre las fechas
    """
    if columna_fecha not in df.columns:
        print(f" Columna '{columna_fecha}' no encontrada")
        return

    print(f"\n ESTADÍSTICAS DETALLADAS DE FECHAS:")

    # Fechas no vacías
    fechas_no_vacias = df[columna_fecha][df[columna_fecha] != '']
    print(f"    Total de fechas no vacías: {len(fechas_no_vacias)}")
    print(f"    Total de fechas vacías: {len(df) - len(fechas_no_vacias)}")

    if len(fechas_no_vacias) > 0:
        # Identificar fechas problemáticas
        fechas_problematicas = fechas_no_vacias[fechas_no_vacias.str.contains('#|ERROR|REF|DIV', na=False)]
        if len(fechas_problematicas) > 0:
            print(f"    Fechas con errores: {len(fechas_problematicas)}")
            print(f"      Ejemplos: {fechas_problematicas.head().tolist()}")

        # Fechas válidas (sin errores)
        fechas_validas = fechas_no_vacias[~fechas_no_vacias.str.contains('#|ERROR|REF|DIV', na=False)]
        if len(fechas_validas) > 0:
            print(f"    Fechas válidas: {len(fechas_validas)}")

            # Intentar obtener rango de fechas válidas
            try:
                fechas_datetime = pd.to_datetime(fechas_validas, dayfirst=True, errors='coerce')
                fechas_datetime = fechas_datetime.dropna()

                if len(fechas_datetime) > 0:
                    print(f"    Rango de fechas válidas: {fechas_datetime.min().strftime('%d/%m/%Y')} - {fechas_datetime.max().strftime('%d/%m/%Y')}")

                    # Distribución por año
                    años = fechas_datetime.dt.year.value_counts().sort_index()
                    print(f"    Distribución por año:")
                    for año, cantidad in años.items():
                        print(f"      {año}: {cantidad} registros")

            except Exception as e:
                print(f"    Error calculando estadísticas de fechas: {e}")

    # Muestra de datos
    print(f"\n Muestra de las primeras 20 fechas:")
    muestra = df[columna_fecha].head(20).tolist()
    for i, fecha in enumerate(muestra, 1):
        print(f"   {i:2d}. {fecha}")

# ============================================================================
# FUNCIÓN PRINCIPAL
# ============================================================================

def obtener_dataframe_historico():
    """
    Obtiene SOLO los datos históricos de la BBDD unificada
    (sin buscar archivos nuevos) - Versión mejorada
    """
    print("=== OBTENIENDO DATAFRAME HISTÓRICO DE AGENDAS PARLAMENTARIAS ===")

    try:
        # Abrir la hoja unificada
        sheet = client.open_by_key(sheet_id_unificado)
        worksheet = sheet.get_worksheet(2)  # Tercera hoja (agendas)

        # Obtener todos los datos
        datos_historicos = worksheet.get_all_values()

        if not datos_historicos or len(datos_historicos) < 2:
            print(" No hay datos históricos disponibles")
            return pd.DataFrame()

        # Crear DataFrame
        encabezados = datos_historicos[0]
        filas = datos_historicos[1:]
        df_historico = pd.DataFrame(filas, columns=encabezados)

        print(f" Datos históricos encontrados: {len(df_historico)} filas")
        print(f" Columnas: {df_historico.columns.tolist()}")

        # Limpiar columnas vacías
        df_historico = df_historico.loc[:, df_historico.columns != '']
        df_historico = df_historico.dropna(axis=1, how='all')

        # Limpiar filas vacías
        filas_antes = len(df_historico)
        df_historico = df_historico.dropna(how='all')

        # Filtrar filas donde columnas principales están vacías
        columnas_principales = ['ID', 'Título de la iniciativa', 'Fecha convocada']
        columnas_existentes = [col for col in columnas_principales if col in df_historico.columns]

        if columnas_existentes:
            mask_vacias = (df_historico[columnas_existentes] == '').all(axis=1)
            df_historico = df_historico[~mask_vacias]

        filas_despues = len(df_historico)
        filas_eliminadas = filas_antes - filas_despues

        if filas_eliminadas > 0:
            print(f" Filas vacías eliminadas: {filas_eliminadas}")

        # Eliminar duplicados por ID si existe la columna
        if 'ID' in df_historico.columns:
            filas_antes_dup = len(df_historico)
            df_historico = df_historico.drop_duplicates(subset='ID', keep='first')
            filas_despues_dup = len(df_historico)
            duplicados_eliminados = filas_antes_dup - filas_despues_dup

            if duplicados_eliminados > 0:
                print(f" Duplicados eliminados (ID): {duplicados_eliminados}")

        # Mostrar estadísticas de fechas ANTES de la corrección
        if 'Fecha convocada' in df_historico.columns:
            print(f"\n ANÁLISIS ANTES DE LA CORRECCIÓN:")
            obtener_estadisticas_fechas(df_historico, 'Fecha convocada')

            # Corregir fechas con la función mejorada
            print(f"\n INICIANDO CORRECCIÓN DE FECHAS:")
            df_historico = corregir_fechas_agendas(df_historico, 'Fecha convocada')

            # Mostrar estadísticas DESPUÉS de la corrección
            print(f"\n ANÁLISIS DESPUÉS DE LA CORRECCIÓN:")
            obtener_estadisticas_fechas(df_historico, 'Fecha convocada')

        # Limpiar texto
        columns_to_replace = ['Sector', 'Marco Geográfico', 'Marco geográfico', 'Proponente']
        for column in columns_to_replace:
            if column in df_historico.columns:
                df_historico[column] = df_historico[column].astype(str).apply(
                    lambda x: x.replace("Castilla_La_Mancha", "Castilla-La Mancha") if "Castilla_La_Mancha" in x else x.replace('_', ' ')
                )

        # Rellenar nulos
        df_historico = df_historico.fillna('')

        print(f"\n DataFrame histórico limpio: {df_historico.shape}")

        # Validación final
        if 'Fecha convocada' in df_historico.columns:
            fechas_con_errores = df_historico['Fecha convocada'].str.contains('#|ERROR|REF|DIV', na=False).sum()
            if fechas_con_errores > 0:
                print(f" Advertencia: Aún quedan {fechas_con_errores} fechas con errores")
            else:
                print(f" Todas las fechas han sido procesadas correctamente")

        return df_historico

    except Exception as e:
        print(f" Error al obtener datos históricos: {e}")
        import traceback
        traceback.print_exc()
        return pd.DataFrame()

def main():
    """Función principal mejorada"""
    print(" OBTENIENDO SOLO DATOS HISTÓRICOS DE AGENDAS PARLAMENTARIAS")
    print("=" * 60)

    dfag_historico = obtener_dataframe_historico()

    if not dfag_historico.empty:
        print(f"\n DATAFRAME HISTÓRICO CREADO:")
        print(f"    Dimensiones: {dfag_historico.shape}")
        print(f"    Columnas: {len(dfag_historico.columns)}")
        print(f"    Filas: {len(dfag_historico)}")

        print("\n Columnas del DataFrame:")
        for i, col in enumerate(dfag_historico.columns, 1):
            print(f"   {i:2d}. {col}")

        # Mostrar datos más representativos
        print("\n Primeras 3 filas:")
        print(dfag_historico.head(3).to_string(max_colwidth=50))

        print("\n Últimas 3 filas:")
        print(dfag_historico.tail(3).to_string(max_colwidth=50))

        # Estadísticas por columnas importantes
        if 'Sector' in dfag_historico.columns:
            print(f"\n Top 10 sectores:")
            sectores = dfag_historico['Sector'].value_counts().head(10)
            for sector, cantidad in sectores.items():
                print(f"   {sector}: {cantidad}")

        if 'Marco geográfico' in dfag_historico.columns:
            print(f"\n Distribución geográfica:")
            geografico = dfag_historico['Marco geográfico'].value_counts().head(10)
            for lugar, cantidad in geografico.items():
                print(f"   {lugar}: {cantidad}")

        print("\n PROCESO COMPLETADO EXITOSAMENTE")
        return dfag_historico
    else:
        print("\n No se pudo crear el DataFrame histórico")
        return None

dfag_historico = main()

if __name__ == "__main__":
    main()

 OBTENIENDO SOLO DATOS HISTÓRICOS DE AGENDAS PARLAMENTARIAS
=== OBTENIENDO DATAFRAME HISTÓRICO DE AGENDAS PARLAMENTARIAS ===
 Datos históricos encontrados: 4103 filas
 Columnas: ['ID', 'Sector', 'Sector2', 'Subsector', 'Tema', 'Legislatura', 'Marco geográfico', 'Fecha convocada', 'Órgano', 'Título de la iniciativa', 'Link', 'Proponente', 'Tipo de iniciativa', 'Color']

 ANÁLISIS ANTES DE LA CORRECCIÓN:

 ESTADÍSTICAS DETALLADAS DE FECHAS:
    Total de fechas no vacías: 4077
    Total de fechas vacías: 26
    Fechas con errores: 18
      Ejemplos: ['#VALUE!', '#VALUE!', '#VALUE!', '#VALUE!', '#VALUE!']
    Fechas válidas: 4059
    Rango de fechas válidas: 09/10/2021 - 21/12/2025
    Distribución por año:
      2021: 1 registros
      2023: 1316 registros
      2024: 1960 registros
      2025: 781 registros

 Muestra de las primeras 20 fechas:
    1. 5/05/2023
    2. 3/05/2023
    3. 3/05/2023
    4. 3/05/2023
    5. 4/05/2023
    6. 4/05/2023
    7. 4/05/2023
    8. 4/05/2023
    9. 4/0

In [ ]:
# # Asegúrate de que la columna esté en formato datetime
# dfag['Fecha convocada'] = pd.to_datetime(dfag['Fecha convocada'], errors='coerce')

# # Filtra por la fecha específica
# fecha_objetivo = pd.Timestamp('2025-07-07')
# filtro = dfag['Fecha convocada'] == fecha_objetivo

# # Cuenta cuántas hay
# cantidad = filtro.sum()
# print(f"Número de registros el 7/7/2025: {cantidad}")

Número de registros el 7/7/2025: 3


## Ahora transformo Agendas parlamentarias a DataFrame pero solamente de aquellas posteriores a cutoff_date

In [ ]:
# ============================================================================
# CONFIGURACIÓN
# ============================================================================

# Parámetros principales
sheet_name = 'Boletines_unificado'
sheet_id_unificado = '1T8l8omO07iWkUyB2tuc6wGf_RJhsgifWUgTb6L2LPN8'
cutoff_date = datetime(2025, 7, 28, 7, 0, 0)  # Fecha de corte

#  CARPETA DE AGENDAS PARLAMENTARIAS
# Reemplaza con el ID de tu carpeta de agendas
folder_id_agendas = '1CCdTfcadG4MadY5Tp665yzhEBAzPQ1Km'  # <<<< CAMBIA ESTO

sheets_to_monitor = []

# Zona horaria
madrid_tz = pytz.timezone('Europe/Madrid')

# Credenciales
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
    "https://www.googleapis.com/auth/spreadsheets"
]

creds = ServiceAccountCredentials.from_json_keyfile_name(
    '/content/drive/Shareddrives/(02 MAD) Iniciativas - AAPP - JP/Automatización_py/test-automation-gsheets-5fa673850271.json',
    scope
)

client = gspread.authorize(creds)
drive_service = build('drive', 'v3', credentials=creds)

# ============================================================================
# FUNCIONES DE CORRECCIÓN DE FECHAS
# ============================================================================

def corregir_fechas_agendas(df, columna_fecha='Fecha convocada'):
    """
    Corrige fechas en la columna de agendas parlamentarias
    DEVUELVE FORMATO DATETIME, NO STRING
    """
    if columna_fecha not in df.columns:
        print(f" Columna '{columna_fecha}' no encontrada")
        return df

    print(f" Procesando fechas en columna '{columna_fecha}'...")

    # Contadores para estadísticas
    errores_value = 0
    fechas_corregidas = 0
    fechas_no_procesadas = 0

    def procesar_fecha(fecha_str):
        nonlocal errores_value, fechas_corregidas, fechas_no_procesadas

        if not fecha_str or fecha_str == '':
            return pd.NaT  # Not a Time para valores vacíos

        fecha_str = str(fecha_str).strip()

        # Manejar errores de Excel/Google Sheets
        if fecha_str in ['#VALUE!', '#ERROR!', '#REF!', '#N/A', '#DIV/0!']:
            errores_value += 1
            return pd.NaT

        try:
            # Intentar formato europeo DD/MM/YYYY
            try:
                fecha_obj = pd.to_datetime(fecha_str, format='%d/%m/%Y', dayfirst=True)
                fechas_corregidas += 1
                return fecha_obj  # DEVOLVER DATETIME, NO STRING
            except:
                pass

            # Intentar parseo automático
            try:
                fecha_obj = pd.to_datetime(fecha_str, dayfirst=True)
                fechas_corregidas += 1
                return fecha_obj  # DEVOLVER DATETIME, NO STRING
            except:
                pass

            # Si no se puede procesar, devolver NaT
            fechas_no_procesadas += 1
            return pd.NaT

        except Exception as e:
            fechas_no_procesadas += 1
            return pd.NaT

    # Aplicar corrección
    df[columna_fecha] = df[columna_fecha].apply(procesar_fecha)

    # Mostrar estadísticas
    print(f"    Fechas corregidas: {fechas_corregidas}")
    print(f"    Errores de fórmula: {errores_value}")
    print(f"    Fechas no procesadas: {fechas_no_procesadas}")

    return df

# ============================================================================
# FUNCIONES PARA BUSCAR Y LEER GOOGLE SHEETS
# ============================================================================

def obtener_sheets_modificados_recientes(folder_id, cutoff_date):
    """
    Busca Google Sheets modificados después de la fecha de corte
    (Misma lógica que el script de boletines autonómicos)
    """
    print(f" Buscando Google Sheets modificados después de {cutoff_date}")

    query = f"'{folder_id}' in parents and mimeType='application/vnd.google-apps.spreadsheet' and trashed=false"

    try:
        results = drive_service.files().list(
            q=query,
            fields="files(id, name, modifiedTime)",
            pageSize=100,
            supportsAllDrives=True,
            includeItemsFromAllDrives=True
        ).execute()

        files = results.get('files', [])
        print(f" Total de Google Sheets encontrados: {len(files)}")

        sheets_recientes = []

        for file in files:
            modified_time_str = file['modifiedTime']
            modified_time = datetime.fromisoformat(modified_time_str.replace('Z', '+00:00'))
            modified_time_local = modified_time.replace(tzinfo=None)

            if modified_time_local >= cutoff_date:
                sheets_recientes.append({
                    'id': file['id'],
                    'name': file['name'],
                    'modified_time': modified_time_local
                })
                print(f"    {file['name']} - Modificado: {modified_time_local}")
            else:
                print(f"    {file['name']} - Modificado: {modified_time_local} (anterior)")

        print(f" Hojas que cumplen criterio: {len(sheets_recientes)}")
        return sheets_recientes

    except Exception as e:
        print(f" Error al buscar hojas: {e}")
        return []

def leer_google_sheet(sheet_id):
    """
    Lee una hoja de Google Sheets y la convierte a DataFrame
    """
    try:
        sheet = client.open_by_key(sheet_id)
        worksheet = sheet.get_worksheet(0)  # Primera hoja

        datos = worksheet.get_all_values()

        if not datos or len(datos) < 2:
            print(f"    Hoja vacía")
            return pd.DataFrame()

        # Crear DataFrame
        encabezados = datos[0]
        filas = datos[1:]
        df = pd.DataFrame(filas, columns=encabezados)

        # Filtrar filas vacías
        df = df.dropna(how='all')

        # Filtrar por columnas principales
        columnas_principales = ['ID', 'Título de la iniciativa', 'Fecha convocada']
        columnas_existentes = [col for col in columnas_principales if col in df.columns]

        if columnas_existentes:
            mask_vacias = (df[columnas_existentes] == '').all(axis=1)
            df = df[~mask_vacias]

        return df

    except Exception as e:
        print(f"    Error al leer hoja {sheet_id}: {e}")
        return pd.DataFrame()

def leer_datos_sheet_especifico(sheet_id, sheet_name="Hoja1"):
    """
    Lee datos de una Google Sheet específica con nombre de hoja
    """
    try:
        print(f" Leyendo datos de Google Sheet: {sheet_id}")

        sheet = client.open_by_key(sheet_id)

        # Intentar diferentes nombres de hoja
        worksheet_names = [sheet_name, "Hoja1", "Sheet1", "Datos"]
        worksheet = None

        for name in worksheet_names:
            try:
                worksheet = sheet.worksheet(name)
                print(f"    Hoja encontrada: {name}")
                break
            except:
                continue

        if worksheet is None:
            worksheet = sheet.get_worksheet(0)
            print(f"    Usando primera hoja: {worksheet.title}")

        datos = worksheet.get_all_records()

        if not datos:
            print("    No se encontraron datos")
            return pd.DataFrame()

        df = pd.DataFrame(datos)
        print(f"    Datos leídos: {df.shape}")

        return df

    except Exception as e:
        print(f" Error leyendo Sheet {sheet_id}: {e}")
        return pd.DataFrame()

# ============================================================================
# FUNCIÓN PRINCIPAL UNIFICADA
# ============================================================================

def obtener_nuevos_datos_agendas():
    """
    Obtiene datos nuevos de agendas parlamentarias
    Combina búsqueda automática por carpeta Y lista manual
    SIN COLUMNAS DE FUENTE EN EL RESULTADO FINAL
    ORDENADOS CRONOLÓGICAMENTE POR FECHA CONVOCADA
    """
    print("=== OBTENIENDO DATOS NUEVOS DE AGENDAS PARLAMENTARIAS ===")
    print(f" Fecha de corte: {cutoff_date}")

    df_nuevos = []

    #  MÉTODO 1: Buscar por carpeta (como boletines autonómicos)
    if folder_id_agendas != 'TU_FOLDER_ID_DE_AGENDAS':
        print(f"\n MÉTODO 1: Buscar en carpeta {folder_id_agendas}")

        sheets_recientes = obtener_sheets_modificados_recientes(folder_id_agendas, cutoff_date)

        for sheet_info in sheets_recientes:
            print(f"\n Procesando: {sheet_info['name']}")

            df_temp = leer_google_sheet(sheet_info['id'])

            if not df_temp.empty:
                df_temp['_source_sheet'] = sheet_info['name']
                df_temp['_source_id'] = sheet_info['id']
                df_nuevos.append(df_temp)
                print(f"    DataFrame creado: {df_temp.shape}")
            else:
                print(f"    DataFrame vacío")

    #  MÉTODO 2: Lista manual (como el script original)
    if sheets_to_monitor:
        print(f"\n MÉTODO 2: Lista manual ({len(sheets_to_monitor)} sheets)")

        for sheet_info in sheets_to_monitor:
            sheet_id = sheet_info['id']
            sheet_name = sheet_info.get('name', 'Desconocida')
            worksheet_name = sheet_info.get('worksheet', 'Hoja1')

            print(f"\n Procesando: {sheet_name}")

            df_temp = leer_datos_sheet_especifico(sheet_id, worksheet_name)

            if not df_temp.empty:
                df_temp['_source_sheet'] = sheet_name
                df_temp['_source_id'] = sheet_id
                df_nuevos.append(df_temp)
                print(f"    DataFrame creado: {df_temp.shape}")
            else:
                print(f"    DataFrame vacío")

    #  VERIFICAR SI HAY DATOS
    if not df_nuevos:
        print("\n No se encontraron datos nuevos")
        print(" Verifica:")
        print("   - folder_id_agendas esté configurado correctamente")
        print("   - sheets_to_monitor tenga IDs válidos")
        print("   - Las fechas de modificación sean posteriores al cutoff_date")
        return pd.DataFrame()

    #  COMBINAR TODOS LOS DATAFRAMES
    print(f"\n Combinando {len(df_nuevos)} DataFrames...")
    dfag_nuevos = pd.concat(df_nuevos, ignore_index=True)

    print(f" DataFrame combinado inicial: {dfag_nuevos.shape}")

    #  ALINEAR COLUMNAS PARA AGENDAS (SIN INCLUIR COLUMNAS DE FUENTE)
    columnas_agendas = ['ID', 'Sector', 'Sector2', 'Subsector', 'Tema', 'Legislatura',
                       'Marco geográfico', 'Fecha convocada', 'Órgano', 'Título de la iniciativa',
                       'Link', 'Proponente', 'Tipo de iniciativa']

    columnas_existentes = [col for col in columnas_agendas if col in dfag_nuevos.columns]
    columnas_faltantes = [col for col in columnas_agendas if col not in dfag_nuevos.columns]

    if columnas_faltantes:
        print(f" Columnas faltantes: {columnas_faltantes}")

    # SOLO MANTENER LAS COLUMNAS DE AGENDAS (ELIMINAR LAS DE FUENTE)
    dfag_nuevos = dfag_nuevos[columnas_existentes]

    #  CORREGIR FECHAS (MANTENER COMO DATETIME)
    if 'Fecha convocada' in dfag_nuevos.columns:
        print(f"\n PROCESANDO FECHAS:")
        dfag_nuevos = corregir_fechas_agendas(dfag_nuevos, 'Fecha convocada')

        # *** NUEVO: ORDENAMIENTO CRONOLÓGICO ***
        print(f"\n ORDENAMIENTO CRONOLÓGICO:")
        filas_antes_ordenar = len(dfag_nuevos)

        # Separar filas con fecha válida y sin fecha
        filas_con_fecha = dfag_nuevos[dfag_nuevos['Fecha convocada'].notna()].copy()
        filas_sin_fecha = dfag_nuevos[dfag_nuevos['Fecha convocada'].isna()].copy()

        if not filas_con_fecha.empty:
            # Ordenar las filas con fecha cronológicamente
            filas_con_fecha = filas_con_fecha.sort_values('Fecha convocada', ascending=True)
            print(f"    Filas con fecha ordenadas: {len(filas_con_fecha)}")

            # Mostrar rango de fechas
            fecha_min = filas_con_fecha['Fecha convocada'].min()
            fecha_max = filas_con_fecha['Fecha convocada'].max()
            print(f"    Rango de fechas: {fecha_min.strftime('%d/%m/%Y')} - {fecha_max.strftime('%d/%m/%Y')}")

        if not filas_sin_fecha.empty:
            print(f"    Filas sin fecha (al final): {len(filas_sin_fecha)}")

        # Recombinar: primero las ordenadas por fecha, luego las sin fecha
        if not filas_con_fecha.empty and not filas_sin_fecha.empty:
            dfag_nuevos = pd.concat([filas_con_fecha, filas_sin_fecha], ignore_index=True)
        elif not filas_con_fecha.empty:
            dfag_nuevos = filas_con_fecha.reset_index(drop=True)
        else:
            dfag_nuevos = filas_sin_fecha.reset_index(drop=True)

        print(f"    DataFrame ordenado: {len(dfag_nuevos)} filas")

    #  LIMPIEZA FINAL
    print(f"\n LIMPIEZA FINAL:")

    # Eliminar duplicados
    if 'ID' in dfag_nuevos.columns:
        filas_antes = len(dfag_nuevos)
        dfag_nuevos = dfag_nuevos.drop_duplicates(subset='ID', keep='first')
        duplicados = filas_antes - len(dfag_nuevos)
        if duplicados > 0:
            print(f"   🧹 Duplicados eliminados: {duplicados}")

    # Rellenar valores nulos (pero NO en la columna de fecha para mantener datetime)
    columnas_texto = [col for col in dfag_nuevos.columns if col != 'Fecha convocada']
    dfag_nuevos[columnas_texto] = dfag_nuevos[columnas_texto].fillna('')

    print(f"    DataFrame final ordenado cronológicamente: {dfag_nuevos.shape}")

    return dfag_nuevos

# ============================================================================
# EJECUCIÓN
# ============================================================================

def main():
    """Función principal"""
    print(" OBTENIENDO DATOS NUEVOS DE AGENDAS PARLAMENTARIAS")
    print("=" * 60)

    # Verificar configuración
    if folder_id_agendas == 'TU_FOLDER_ID_DE_AGENDAS' and not sheets_to_monitor:
        print(" CONFIGURACIÓN REQUERIDA:")
        print("Opción 1: Configurar folder_id_agendas = 'ID_DE_TU_CARPETA'")
        print("Opción 2: Configurar sheets_to_monitor = [{'id': '...', 'name': '...'}]")
        print("Opción 3: Configurar ambos")
        return None

    # Obtener datos
    dfag_nuevos = obtener_nuevos_datos_agendas()

    if not dfag_nuevos.empty:
        print(f"\n ÉXITO: DataFrame creado con {len(dfag_nuevos)} nuevas filas")
        print(f" Columnas: {dfag_nuevos.columns.tolist()}")
        print(f" Forma: {dfag_nuevos.shape}")

        # Mostrar muestra con fechas para verificar orden
        print("\n Muestra del DataFrame (primeras 5 filas):")
        if 'Fecha convocada' in dfag_nuevos.columns:
            columnas_muestra = ['Fecha convocada', 'Título de la iniciativa']
            columnas_disponibles = [col for col in columnas_muestra if col in dfag_nuevos.columns]
            print(dfag_nuevos[columnas_disponibles].head())
        else:
            print(dfag_nuevos.head())

        # Estadísticas por fuente
        if '_source_sheet' in dfag_nuevos.columns:
            print(f"\n Datos por fuente:")
            fuentes = dfag_nuevos['_source_sheet'].value_counts()
            for fuente, cantidad in fuentes.items():
                print(f"   {fuente}: {cantidad}")

        # Verificar ordenamiento cronológico
        if 'Fecha convocada' in dfag_nuevos.columns:
            fechas_validas = dfag_nuevos[dfag_nuevos['Fecha convocada'].notna()]
            if not fechas_validas.empty:
                print(f"\n  VERIFICACIÓN DE ORDEN CRONOLÓGICO:")
                print(f"    Primera fecha: {fechas_validas.iloc[0]['Fecha convocada'].strftime('%d/%m/%Y')}")
                print(f"    Última fecha: {fechas_validas.iloc[-1]['Fecha convocada'].strftime('%d/%m/%Y')}")

        return dfag_nuevos
    else:
        print("\n No se encontraron datos nuevos")
        return None

dfag_nuevos = main()

# Verificar que se creó correctamente
if dfag_nuevos is not None:
    print(f"Variable dfag_nuevos creada exitosamente: {dfag_nuevos.shape}")
else:
    print("dfag_nuevos es None - no se encontraron datos")
    dfag_nuevos = pd.DataFrame()

print("\n" + "=" * 60)

 OBTENIENDO DATOS NUEVOS DE AGENDAS PARLAMENTARIAS
=== OBTENIENDO DATOS NUEVOS DE AGENDAS PARLAMENTARIAS ===
 Fecha de corte: 2025-07-28 07:00:00

 MÉTODO 1: Buscar en carpeta 1CCdTfcadG4MadY5Tp665yzhEBAzPQ1Km
 Buscando Google Sheets modificados después de 2025-07-28 07:00:00
 Total de Google Sheets encontrados: 3
    250728 Agendas Parlamentarias - Modificado: 2025-07-28 07:39:17.601000
    250721 Agendas Parlamentarias - Modificado: 2025-07-21 08:25:53.874000 (anterior)
    250714 Agendas Parlamentarias - Modificado: 2025-07-16 15:22:39.715000 (anterior)
 Hojas que cumplen criterio: 1

 Procesando: 250728 Agendas Parlamentarias
    DataFrame creado: (3, 15)

 Combinando 1 DataFrames...
 DataFrame combinado inicial: (3, 15)

 PROCESANDO FECHAS:
 Procesando fechas en columna 'Fecha convocada'...
    Fechas corregidas: 3
    Errores de fórmula: 0
    Fechas no procesadas: 0

 ORDENAMIENTO CRONOLÓGICO:
    Filas con fecha ordenadas: 3
    Rango de fechas: 28/07/2025 - 28/07/2025
    Data

In [ ]:
dfag_nuevos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   ID                       3 non-null      object        
 1   Sector                   3 non-null      object        
 2   Sector2                  3 non-null      object        
 3   Subsector                3 non-null      object        
 4   Tema                     3 non-null      object        
 5   Legislatura              3 non-null      object        
 6   Marco geográfico         3 non-null      object        
 7   Fecha convocada          3 non-null      datetime64[ns]
 8   Órgano                   3 non-null      object        
 9   Título de la iniciativa  3 non-null      object        
 10  Link                     3 non-null      object        
 11  Proponente               3 non-null      object        
 12  Tipo de iniciativa       3 non-null     

# <u>EXPORTACIÓN a **BigQuery**</u>:
## * **Depuración** de los dataframe para que coincidan con estructura BigQuery
## * Creación del proyecto con credenciales + service account + permisos: [[link]](https://cloud.google.com/bigquery/docs/use-service-accounts?hl=es-419)
## * **Creación de tablas** 1 x 1

## 1.  Boletines parlamentarios

In [ ]:
print(dfp_historico.shape)
print(dfp_historico.head())

(18210, 12)
  ID             Sector Sector2           Subsector Tema Legislatura  \
0  1       Alimentación                                          XIV   
1  2  Economía Circular                                          XIV   
2  3              Salud              Salud - Cáncer              XIV   
3  4           Primario          Primario - Cárnico               XI   
4  5              Pesca                                           XI   

  Marco geográfico                            Título de la iniciativa  \
0           España  Proposición de Ley en apoyo del sistema alimen...   
1           España  Proposición no de Ley sobre impulso de la rena...   
2           España  Moción ante el Pleno por la que se insta al Go...   
3        Andalucía  Solicitud de comparecencia de la consejera de ...   
4        Andalucía  Proposición no de ley relativa a defensa de la...   

                                                Link Fecha publicación  \
0  https://www.congreso.es/public_oficiale

In [15]:
print(dfp_nuevos.shape)
print(dfp_nuevos.head())

(23, 11)
      ID          Sector Subsector Tema Legislatura Marco geográfico  \
0  18211        Primario                         XV           España   
1  18212        Primario                         XV           España   
2  18213        Primario                         XV           España   
3  18214  Digitalización                        XII        Andalucía   
4  18215        Primario                        XII        Andalucía   

                             Título de la iniciativa  \
0  Proposición no de Ley relativa a la creación d...   
1  Proposición no de Ley relativa a la agricultur...   
2  Proposición no de ley relativa a la regulación...   
3  Pregunta relativa al Centro de Inteligencia Ar...   
4  Pregunta relativa a las actuaciones previstas ...   

                                                Link Fecha publicación  \
0  https://www.congreso.es/public_oficiales/L15/C...        30/07/2025   
1  https://www.congreso.es/public_oficiales/L15/C...        30/07/2025   

In [16]:
dfp_nuevos.columns = (
    dfp_nuevos.columns
    .str.strip()                          # elimina espacios adelante/atrás
    .str.lower()                          # minúsculas
    .str.replace(' ', '_')                # espacios por guiones bajos
    .str.replace(r'[áàäâ]', 'a', regex=True)
    .str.replace(r'[éèëê]', 'e', regex=True)
    .str.replace(r'[íìïî]', 'i', regex=True)
    .str.replace(r'[óòöô]', 'o', regex=True)
    .str.replace(r'[úùüû]', 'u', regex=True)
    .str.replace(r'ñ', 'n', regex=True)
    .str.replace(r'[^\w]', '', regex=True)  # elimina cualquier otro carácter raro
)


# Convertir valores nulos a None para BigQuery
dfp_nuevos = dfp_nuevos.where(pd.notnull(dfp_nuevos), None)

#dfp_nuevos['id'] = dfp_nuevos['id'].astype(str)

dfp_nuevos['id'] = dfp_nuevos['id'].astype(int)

# #Convertir fechas a datetime explícitamente
# dfp_historico['fecha_publicacion'] = pd.to_datetime(dfp_historico['fecha_publicacion'], format='%d/%m/%Y', dayfirst=True, errors='coerce')


In [ ]:
# dfp.columns = (
#     dfp.columns
#     .str.strip()                          # elimina espacios adelante/atrás
#     .str.lower()                          # minúsculas
#     .str.replace(' ', '_')                # espacios por guiones bajos
#     .str.replace(r'[áàäâ]', 'a', regex=True)
#     .str.replace(r'[éèëê]', 'e', regex=True)
#     .str.replace(r'[íìïî]', 'i', regex=True)
#     .str.replace(r'[óòöô]', 'o', regex=True)
#     .str.replace(r'[úùüû]', 'u', regex=True)
#     .str.replace(r'ñ', 'n', regex=True)
#     .str.replace(r'[^\w]', '', regex=True)  # elimina cualquier otro carácter raro
# )

# #Convertir fechas a datetime explícitamente
# dfp['fecha_publicacion'] = pd.to_datetime(dfp['fecha_publicacion'], format='%d/%m/%Y', dayfirst=True, errors='coerce')

# # Convertir valores nulos a None para BigQuery
# dfp = dfp.where(pd.notnull(dfp), None)

# dfp['id'] = dfp['id'].astype(str)

# if 'id' in dfp.columns:
#     dfp['id'] = pd.to_numeric(dfp['id'], errors='coerce', downcast='integer')

# dfp = dfp.where(pd.notnull(dfp), None)

In [ ]:
# # Ver fechas nulas después del parsing
# print(dfp['fecha_publicacion'].isnull().sum())

6798


In [ ]:
# Eliminar columnas sin nombre
dfp_historico = dfp_historico.loc[:, dfp_historico.columns != '']


In [17]:
dfp_nuevos = dfp_nuevos.loc[:, dfp_nuevos.columns != '']

In [ ]:
# dfp = dfp.astype(str).fillna('')


In [ ]:
# dfp_nuevos = dfp_nuevos.astype(str).fillna('')


In [ ]:
print(dfp_historico.columns.tolist())

['id', 'sector', 'sector2', 'subsector', 'tema', 'legislatura', 'marco_geografico', 'titulo_de_la_iniciativa', 'link', 'fecha_publicacion', 'proponente', 'tipo_de_iniciativa']


In [18]:
print(dfp_nuevos.columns.tolist())

['id', 'sector', 'subsector', 'tema', 'legislatura', 'marco_geografico', 'titulo_de_la_iniciativa', 'link', 'fecha_publicacion', 'proponente', 'tipo_de_iniciativa']


# Tratamiento de la columna de fechas: la convierto a datetime (formato día/mes/año)

In [19]:
dfp_nuevos['fecha_publicacion'] = pd.to_datetime(
    dfp_nuevos['fecha_publicacion'],
    dayfirst=True,
    errors='coerce',
    )

# # Cambiar el formato de la fecha a 'día/mes/año'
# dfp_nuevos['fecha_publicacion'] = dfp_nuevos['fecha_publicacion'].dt.strftime('%d/%m/%Y')

# # Eliminar filas con fechas inválidas
# dfp_historico = dfp_historico[dfp_historico['fecha_publicacion'].notna()]

# Verifica si la conversión a datetime fue exitosa
print(dfp_nuevos['fecha_publicacion'].dtype)  # Debería ser datetime64[ns]

datetime64[ns]


In [20]:
print(dfp_nuevos['fecha_publicacion'].apply(type).value_counts())


fecha_publicacion
<class 'pandas._libs.tslibs.timestamps.Timestamp'>    23
Name: count, dtype: int64


In [21]:
dfp_nuevos

,id,sector,subsector,tema,legislatura,marco_geografico,titulo_de_la_iniciativa,link,fecha_publicacion,proponente,tipo_de_iniciativa
0,18211,Primario,,,XV,España,Proposición no de Ley relativa a la creación d...,https://www.congreso.es/public_oficiales/L15/C...,2025-07-30,Congreso - Grupo Plural Sumar,Proposición no de Ley en Comisión
1,18212,Primario,,,XV,España,Proposición no de Ley relativa a la agricultur...,https://www.congreso.es/public_oficiales/L15/C...,2025-07-30,Congreso - Grupo Plural Sumar,Proposición no de Ley en Comisión
2,18213,Primario,,,XV,España,Proposición no de ley relativa a la regulación...,https://www.congreso.es/public_oficiales/L15/C...,2025-07-30,Congreso - Grupo Plural Sumar,Proposición no de Ley en Comisión
3,18214,Digitalización,,,XII,Andalucía,Pregunta relativa al Centro de Inteligencia Ar...,https://www.parlamentodeandalucia.es/webdinami...,2025-07-30,Andalucía - Grupo Mixto - Por Andalucía,Pregunta en Comisión
4,18215,Primario,,,XII,Andalucía,Pregunta relativa a las actuaciones previstas ...,https://www.parlamentodeandalucia.es/webdinami...,2025-07-30,Andalucía - Grupo VOX,Pregunta en Comisión
5,18216,Primario,,,XII,Andalucía,Pregunta relativa a los daños producidos en la...,https://www.parlamentodeandalucia.es/webdinami...,2025-07-30,Andalucía - Grupo VOX,Pregunta en Comisión
6,18217,Empresa,,,XII,Andalucía,Pregunta relativa a las startups,https://www.parlamentodeandalucia.es/webdinami...,2025-07-30,Andalucía - Grupo VOX,Pregunta en Comisión
7,18218,Turismo,,,XII,Andalucía,Pregunta relativa al compromiso andaluz de acc...,https://www.parlamentodeandalucia.es/webdinami...,2025-07-30,Andalucía - Grupo Popular,Pregunta en Comisión
8,18219,Turismo,,,XII,Andalucía,Pregunta relativa a la innovación turística (N...,https://www.parlamentodeandalucia.es/webdinami...,2025-07-30,Andalucía - Grupo Popular,Pregunta en Comisión
9,18220,Turismo,,,XII,Andalucía,Pregunta relativa al turismo MICE,https://www.parlamentodeandalucia.es/webdinami...,2025-07-30,Andalucía - Grupo Popular,Pregunta en Comisión


In [22]:
dfp_nuevos.dtypes

,0
id,int64
sector,object
subsector,object
tema,object
legislatura,object
marco_geografico,object
titulo_de_la_iniciativa,object
link,object
fecha_publicacion,datetime64[ns]
proponente,object


In [ ]:
# dfp.dtypes

## 1.1 Exportación de boletines parlamentarios a BigQuery

In [ ]:
# from google.cloud import bigquery

# SERVICE_ACCOUNT_FILE = "/content/drive/Shareddrives/(02 MAD) Iniciativas - AAPP - JP/Automatización_py/Exportación_BigQuery/emerald-vigil-438909-s2-74505858a60d.json" #el nombre del json específico

# PROJECT_ID = "emerald-vigil-438909-s2"
# DATASET_ID = "BBDD_unificada"
# TABLE_NAME = "Boletines parlamentarios" #¿debo colocar el nombre de la tabla unificada, o de cada una de ellos?

# client = bigquery.Client.from_service_account_json(SERVICE_ACCOUNT_FILE)

# QUERY = f"SELECT * FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_NAME}` LIMIT 100"

# try:
#   df = client.query(QUERY).to_dataframe()
# except Exception as e:
#   print("Error al cargar datos de BigQuery", e)

Error al cargar datos de BigQuery 404 Not found: Table emerald-vigil-438909-s2:BBDD_unificada.Boletines parlamentarios was not found in location europe-southwest1; reason: notFound, message: Not found: Table emerald-vigil-438909-s2:BBDD_unificada.Boletines parlamentarios was not found in location europe-southwest1

Location: europe-southwest1
Job ID: e9319a95-7092-41c0-bcf7-cc98dbb1df9a



In [23]:
from google.cloud import bigquery
from google.oauth2 import service_account

# Ruta al archivo JSON de credenciales
SERVICE_ACCOUNT_FILE = '/content/drive/Shareddrives/(02 MAD) Iniciativas - AAPP - JP/Automatización_py/Exportación_BigQuery/emerald-vigil-438909-s2-74505858a60d.json'

# Cargar credenciales desde archivo
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE)

# Configuración del proyecto y cliente BigQuery
project_id = 'emerald-vigil-438909-s2'
client = bigquery.Client(credentials=credentials, project=project_id)

# Dataset y tabla
dataset_id = 'BBDD_unificada'
table_id = 'Boletines_parlamentarios'  # sin espacios
table_ref = f"{project_id}.{dataset_id}.{table_id}"

job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_APPEND'
    # write_disposition='WRITE_TRUNCATE',  # caso a tabela nao exista en caso a tabela exista? cómo faz?
    # autodetect=True,  # caso a tabela nao exista
)

# Suponiendo que tu DataFrame ya se llama df_boletines_p
job = client.load_table_from_dataframe(dfp_nuevos, table_ref, job_config=job_config)
job.result()  # Espera a que se complete el job

print(f"✅ Tabla cargada correctamente: {table_ref}")


✅ Tabla cargada correctamente: emerald-vigil-438909-s2.BBDD_unificada.Boletines_parlamentarios


## 2. Boletines autonómicos

In [ ]:
# print(dfa.shape)
# print(dfa.head())

In [24]:
print(dfa_nuevos.shape)
print(dfa_nuevos.head())

(66, 12)
      ID     Sector   Sector2 Sector3          Subsector Tema  \
0  10992    Turismo                                             
1  10993     Sequía  Primario          Sequía - Regadíos        
2  10994   Primario                                             
3  10995      Salud                                             
4  10996  Movilidad                                             

  Marco Geográfico Fecha de publicación  \
0        Andalucía           30/07/2025   
1        Andalucía           30/07/2025   
2        Andalucía           30/07/2025   
3        Andalucía           30/07/2025   
4        Andalucía           30/07/2025   

                                   Título iniciativa  \
0  Extracto de la Orden de 30 de junio de 2025, p...   
1  Orden de 23 de julio de 2025, por la que se ap...   
2  Resolución de 18 de julio de 2025, de la Direc...   
3  Orden de 24 de julio de 2025, por la que se ap...   
4  Resolución de 18 de julio de 2025, de la Direc...   

    

In [ ]:
# # 1. Revisar columnas duplicadas
# if dfa.columns.duplicated().any():
#     print("Columnas duplicadas:", dfa.columns[dfa.columns.duplicated()])

# # 2. Asegurar nombres válidos
# dfa.columns = [col.strip().replace(' ', '_').replace('-', '_') for col in dfa.columns]

# # 3. Asegurar tipos compatibles
# print(dfa.dtypes)

# # 4. Asegurarte de que no haya objetos raros (como listas o diccionarios)
# for col in dfa.columns:
#     if dfa[col].apply(lambda x: isinstance(x, (list, dict))).any():
#         print(f"Columna con valores no válidos para BigQuery: {col}")


In [ ]:
# dfa.columns = (
#     dfa.columns
#     .str.strip()                          # elimina espacios adelante/atrás
#     .str.lower()                          # minúsculas
#     .str.replace(' ', '_')                # espacios por guiones bajos
#     .str.replace(r'[áàäâ]', 'a', regex=True)
#     .str.replace(r'[éèëê]', 'e', regex=True)
#     .str.replace(r'[íìïî]', 'i', regex=True)
#     .str.replace(r'[óòöô]', 'o', regex=True)
#     .str.replace(r'[úùüû]', 'u', regex=True)
#     .str.replace(r'ñ', 'n', regex=True)
#     .str.replace(r'[^\w]', '', regex=True)  # elimina cualquier otro carácter raro
# )

# # Convertir fechas a datetime explícitamente
# dfa['fecha_de_publicacion'] = pd.to_datetime(dfa['fecha_de_publicacion'], format='%d/%m/%Y', dayfirst=True, errors='coerce')



In [25]:
dfa_nuevos.columns = (
    dfa_nuevos.columns
    .str.strip()                          # elimina espacios adelante/atrás
    .str.lower()                          # minúsculas
    .str.replace(' ', '_')                # espacios por guiones bajos
    .str.replace(r'[áàäâ]', 'a', regex=True)
    .str.replace(r'[éèëê]', 'e', regex=True)
    .str.replace(r'[íìïî]', 'i', regex=True)
    .str.replace(r'[óòöô]', 'o', regex=True)
    .str.replace(r'[úùüû]', 'u', regex=True)
    .str.replace(r'ñ', 'n', regex=True)
    .str.replace(r'[^\w]', '', regex=True)  # elimina cualquier otro carácter raro que no se corresponda
)


# Convertir valores nulos a None para BigQuery
dfa_nuevos = dfa_nuevos.where(pd.notnull(dfa_nuevos), None)

dfa_nuevos['id'] = dfa_nuevos['id'].astype(str)

# Convertir fechas a datetime explícitamente
dfa_nuevos['fecha_de_publicacion'] = pd.to_datetime(dfa_nuevos['fecha_de_publicacion'], format='%d/%m/%Y', dayfirst=True, errors='coerce')

# # Convertir valores nulos a None para BigQuery
# dfa = dfa.where(pd.notnull(dfa), None)


In [ ]:
# dfa = dfa.fillna('')
# print(dfa)


In [26]:
dfa_nuevos = dfa_nuevos.fillna('')
print(dfa_nuevos)

       id        sector   sector2 sector3                      subsector tema  \
0   10992       Turismo                                                         
1   10993        Sequía  Primario                      Sequía - Regadíos        
2   10994      Primario                                                         
3   10995         Salud                                                         
4   10996     Movilidad                                                         
..    ...           ...       ...     ...                            ...  ...   
61  11053  Alimentación  Comercio                                               
62  11054      Primario                                                         
63  11055  Farmacéutico                    Farmacéutico - Fitosanitarios        
64  11056  Alimentación                                                         
65  11057  Alimentación     Salud                                               

   marco_geografico fecha_d

In [ ]:
# Verificar el tipo de datos de la columna 'Fecha de publicación'
# tipo_columna = dfa['fecha_de_publicacion'].dtype
# print(f"El tipo de la columna 'Fecha de publicación' es: {tipo_columna}")


In [ ]:
# Eliminar columnas sin nombre
# dfa = dfa.loc[:, dfa.columns != '']

In [27]:
# Verificar el tipo de datos de la columna 'Fecha de publicación'
tipo_columna = dfa_nuevos['fecha_de_publicacion'].dtype
print(f"El tipo de la columna 'Fecha de publicación' es: {tipo_columna}")

El tipo de la columna 'Fecha de publicación' es: datetime64[ns]


In [28]:
dfa_nuevos = dfa_nuevos.loc[:, dfa_nuevos.columns != '']

In [ ]:
# print(dfa.columns.tolist())

In [29]:
print(dfa_nuevos.columns.tolist())

['id', 'sector', 'sector2', 'sector3', 'subsector', 'tema', 'marco_geografico', 'fecha_de_publicacion', 'titulo_iniciativa', 'link', 'proponente', 'subgrupo']


In [30]:
dfa_nuevos.head()

,id,sector,sector2,sector3,subsector,tema,marco_geografico,fecha_de_publicacion,titulo_iniciativa,link,proponente,subgrupo
0,10992,Turismo,,,,,Andalucía,2025-07-30,"Extracto de la Orden de 30 de junio de 2025, p...",chrome-extension://efaidnbmnnnibpcajpcglclefin...,Gobierno de Andalucía,Consejería de Sostenibilidad y Medio Ambiente
1,10993,Sequía,Primario,,Sequía - Regadíos,,Andalucía,2025-07-30,"Orden de 23 de julio de 2025, por la que se ap...",chrome-extension://efaidnbmnnnibpcajpcglclefin...,Gobierno de Andalucía,"Consejería de Agricultura, Pesca, Agua y Desar..."
2,10994,Primario,,,,,Andalucía,2025-07-30,"Resolución de 18 de julio de 2025, de la Direc...",chrome-extension://efaidnbmnnnibpcajpcglclefin...,Gobierno de Andalucía,"Consejería de Agricultura, Pesca, Agua y Desar..."
3,10995,Salud,,,,,Andalucía,2025-07-30,"Orden de 24 de julio de 2025, por la que se ap...",chrome-extension://efaidnbmnnnibpcajpcglclefin...,Gobierno de Andalucía,Consejería de Salud y Consumo
4,10996,Movilidad,,,,,Andalucía,2025-07-30,"Resolución de 18 de julio de 2025, de la Direc...",chrome-extension://efaidnbmnnnibpcajpcglclefin...,Gobierno de Andalucía,"Consejería de Fomento, Articulación del Territ..."


In [ ]:
# from datetime import datetime

# fecha_corte = datetime(2025, 5, 31)
# dfa['Fecha de publicación'] = pd.to_datetime(dfa['Fecha de publicación'], format='%d/%m/%Y', errors='coerce')

In [ ]:
# fechas_mal = dfa[dfa['Fecha de publicación'] > datetime(2025, 5, 31)]
# print(f"Cantidad de fechas posteriores a mayo de 2025: {len(fechas_mal)}")


Cantidad de fechas posteriores a mayo de 2025: 0


## 2.1 Exportación de boletines autonómicos a BigQuery

In [ ]:
# from google.cloud import bigquery

# SERVICE_ACCOUNT_FILE = "/content/drive/Shareddrives/(02 MAD) Iniciativas - AAPP - JP/Automatización_py/Exportación_BigQuery/emerald-vigil-438909-s2-74505858a60d.json" #el nombre del json específico

# PROJECT_ID = "emerald-vigil-438909-s2"
# DATASET_ID = "BBDD_unificada"
# TABLE_NAME = "Boletines autonómicos" #¿debo colocar el nombre de la tabla unificada, o de cada una de ellos?

# client = bigquery.Client.from_service_account_json(SERVICE_ACCOUNT_FILE)

# QUERY = f"SELECT * FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_NAME}` LIMIT 100"

# try:
#   df = client.query(QUERY).to_dataframe()
# except Exception as e:
#   print("Error al cargar datos de BigQuery", e)

Error al cargar datos de BigQuery 404 Not found: Table emerald-vigil-438909-s2:BBDD_unificada.Boletines autonómicos was not found in location europe-southwest1; reason: notFound, message: Not found: Table emerald-vigil-438909-s2:BBDD_unificada.Boletines autonómicos was not found in location europe-southwest1

Location: europe-southwest1
Job ID: e93f2f99-4fcf-4172-a5d3-d1497a067712



In [31]:
from google.cloud import bigquery
from google.oauth2 import service_account

SERVICE_ACCOUNT_FILE = '/content/drive/Shareddrives/(02 MAD) Iniciativas - AAPP - JP/Automatización_py/Exportación_BigQuery/emerald-vigil-438909-s2-74505858a60d.json'

credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE)

project_id = 'emerald-vigil-438909-s2'
client = bigquery.Client(credentials=credentials, project=project_id)

dataset_id = 'BBDD_unificada'
table_id = 'Boletines_autonómicos'
table_ref = f"{project_id}.{dataset_id}.{table_id}"

job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_APPEND'
    # write_disposition='WRITE_TRUNCATE',  # caso a tabela nao exista
    # autodetect=True,  # caso a tabela nao exista
)

job = client.load_table_from_dataframe(dfa_nuevos, table_ref, job_config=job_config)
job.result()

print(f"✅ Tabla cargada correctamente: {table_ref}")


✅ Tabla cargada correctamente: emerald-vigil-438909-s2.BBDD_unificada.Boletines_autonómicos


## 3. Agendas parlamentarias

In [ ]:
print(dfag_historico.shape)
print(dfag_historico.head())

(4103, 14)
  ID                Sector         Sector2         Subsector Tema Legislatura  \
0  1             Industria   Medioambiente                                XIV   
1  2  Financiero y seguros                                                XIV   
2  3             Industria         Empresa                                XII   
3  4            Innovación  Digitalización  Innovación - IDI              XII   
4  5               Consumo           Otros                                XII   

  Marco geográfico Fecha convocada                     Órgano  \
0           España      05/05/2023  Congreso de los Diputados   
1           España      03/05/2023       Comisión de Hacienda   
2        Andalucía      03/05/2023                      Pleno   
3        Andalucía      03/05/2023                      Pleno   
4        Andalucía      04/05/2023                      Pleno   

                             Título de la iniciativa  \
0  Jornada Industria y Descarbonización, a solici...   

In [ ]:
print(dfag_nuevos.shape)
print(dfag_nuevos.head())

(3, 13)
     ID   Sector Sector2 Subsector Tema Legislatura Marco geográfico  \
0  4113  Energía                                 XI         Canarias   
1  4114  Energía                                 XI         Canarias   
2  4115  Energía                                 XI         Canarias   

  Fecha convocada                            Órgano  \
0      2025-07-28  Comisión de Transición Ecológica   
1      2025-07-28  Comisión de Transición Ecológica   
2      2025-07-28  Comisión de Transición Ecológica   

                             Título de la iniciativa  \
0  sobre las propuestas aprobadas por el Minister...   
1  el porcentaje de penetración actual de energía...   
2  si considera que centrarse solo en energías re...   

                                                Link  \
0  https://www.parcan.es/reuniones/asuntos.py?ID_...   
1  https://www.parcan.es/reuniones/asuntos.py?ID_...   
2  https://www.parcan.es/reuniones/asuntos.py?ID_...   

                           Propo

In [ ]:
dfag_nuevos.columns = (
    dfag_nuevos.columns
    .str.strip()                          # elimina espacios adelante/atrás
    .str.lower()                          # minúsculas
    .str.replace(' ', '_')                # espacios por guiones bajos
    .str.replace(r'[áàäâ]', 'a', regex=True)
    .str.replace(r'[éèëê]', 'e', regex=True)
    .str.replace(r'[íìïî]', 'i', regex=True)
    .str.replace(r'[óòöô]', 'o', regex=True)
    .str.replace(r'[úùüû]', 'u', regex=True)
    .str.replace(r'ñ', 'n', regex=True)
    .str.replace(r'[^\w]', '', regex=True)  # elimina cualquier otro carácter raro
)


# Convertir valores nulos a None para BigQuery
dfag_nuevos = dfag_nuevos.where(pd.notnull(dfag_nuevos), None)

dfag_nuevos['id'] = dfag_nuevos['id'].astype(str)

# Convertir fechas a datetime explícitamente
dfag_nuevos['fecha_convocada'] = pd.to_datetime(dfag_nuevos['fecha_convocada'], format='%Y-%m-%d', dayfirst=True, errors='coerce')
dfag_nuevos['fecha_convocada'] = dfag_nuevos['fecha_convocada'].dt.strftime('%Y-%m-%d')

In [ ]:
# dfag_nuevos.columns = (
#     dfag_nuevos.columns
#     .str.strip()
#     .str.lower()
#     .str.replace(' ', '_')
#     .str.replace(r'[^\w_]', '', regex=True)
# )

# dfag_nuevos["fecha_convocada"] = dfag_nuevos["fecha_convocada"].where(
#     dfag_nuevos["fecha_convocada"].notnull(), None
# )

In [ ]:
dfag_nuevos = dfag_nuevos.fillna('')
print(dfag_nuevos)

     id   sector sector2 subsector tema legislatura marco_geografico  \
0  4113  Energía                                 XI         Canarias   
1  4114  Energía                                 XI         Canarias   
2  4115  Energía                                 XI         Canarias   

  fecha_convocada                            organo  \
0      2025-07-28  Comisión de Transición Ecológica   
1      2025-07-28  Comisión de Transición Ecológica   
2      2025-07-28  Comisión de Transición Ecológica   

                             titulo_de_la_iniciativa  \
0  sobre las propuestas aprobadas por el Minister...   
1  el porcentaje de penetración actual de energía...   
2  si considera que centrarse solo en energías re...   

                                                link  \
0  https://www.parcan.es/reuniones/asuntos.py?ID_...   
1  https://www.parcan.es/reuniones/asuntos.py?ID_...   
2  https://www.parcan.es/reuniones/asuntos.py?ID_...   

                           proponente   

In [ ]:
dfag_nuevos = dfag_nuevos.fillna('')
print(dfag_nuevos)

     id   sector sector2 subsector tema legislatura marco_geografico  \
0  4113  Energía                                 XI         Canarias   
1  4114  Energía                                 XI         Canarias   
2  4115  Energía                                 XI         Canarias   

  fecha_convocada                            organo  \
0      2025-07-28  Comisión de Transición Ecológica   
1      2025-07-28  Comisión de Transición Ecológica   
2      2025-07-28  Comisión de Transición Ecológica   

                             titulo_de_la_iniciativa  \
0  sobre las propuestas aprobadas por el Minister...   
1  el porcentaje de penetración actual de energía...   
2  si considera que centrarse solo en energías re...   

                                                link  \
0  https://www.parcan.es/reuniones/asuntos.py?ID_...   
1  https://www.parcan.es/reuniones/asuntos.py?ID_...   
2  https://www.parcan.es/reuniones/asuntos.py?ID_...   

                           proponente   

In [ ]:
# Eliminar columnas sin nombre
dfag_nuevos = dfag_nuevos.loc[:, dfag_nuevos.columns != '']

In [ ]:
# Eliminar columnas sin nombre
dfag_nuevos = dfag_nuevos.loc[:, dfag_nuevos.columns != '']

In [ ]:
print(dfag.columns.tolist())

['id', 'sector', 'sector2', 'subsector', 'tema', 'legislatura', 'marco_geografico', 'fecha_convocada', 'organo', 'titulo_de_la_iniciativa', 'link', 'proponente', 'tipo_de_iniciativa', 'color']


In [ ]:
print(dfag_nuevos.columns.tolist())

['id', 'sector', 'sector2', 'subsector', 'tema', 'legislatura', 'marco_geografico', 'fecha_convocada', 'organo', 'titulo_de_la_iniciativa', 'link', 'proponente', 'tipo_de_iniciativa']


In [ ]:
dfag_nuevos.tail()

,id,sector,sector2,subsector,tema,legislatura,marco_geografico,fecha_convocada,organo,titulo_de_la_iniciativa,link,proponente,tipo_de_iniciativa
0,4113,Energía,,,,XI,Canarias,2025-07-28,Comisión de Transición Ecológica,sobre las propuestas aprobadas por el Minister...,https://www.parcan.es/reuniones/asuntos.py?ID_...,Canarias - Grupo Socialista,Comparecencia del Gobierno en Comisión
1,4114,Energía,,,,XI,Canarias,2025-07-28,Comisión de Transición Ecológica,el porcentaje de penetración actual de energía...,https://www.parcan.es/reuniones/asuntos.py?ID_...,Canarias - Grupo Coalición Canaria,Pregunta en Comisión
2,4115,Energía,,,,XI,Canarias,2025-07-28,Comisión de Transición Ecológica,si considera que centrarse solo en energías re...,https://www.parcan.es/reuniones/asuntos.py?ID_...,Canarias - Grupo VOX,Pregunta en Comisión


In [ ]:
dfag_nuevos.dtypes

,0
id,object
sector,object
sector2,object
subsector,object
tema,object
legislatura,object
marco_geografico,object
fecha_convocada,object
organo,object
titulo_de_la_iniciativa,object


## 3.1 Exportación de Agendas parlamentarias a BigQuery

In [ ]:
# from google.cloud import bigquery

# SERVICE_ACCOUNT_FILE = "/content/drive/Shareddrives/(02 MAD) Iniciativas - AAPP - JP/Automatización_py/Exportación_BigQuery/emerald-vigil-438909-s2-74505858a60d.json" #el nombre del json específico

# PROJECT_ID = "emerald-vigil-438909-s2"
# DATASET_ID = "BBDD_unificada"
# TABLE_NAME = "Agendas parlamentarias" #¿debo colocar el nombre de la tabla unificada, o de cada una de ellos?

# client = bigquery.Client.from_service_account_json(SERVICE_ACCOUNT_FILE)

# QUERY = f"SELECT * FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_NAME}` LIMIT 100"

# try:
#   dfag = client.query(QUERY).to_dataframe()
# except Exception as e:
#   print("Error al cargar datos de BigQuery", e)

Error al cargar datos de BigQuery 404 Not found: Table emerald-vigil-438909-s2:BBDD_unificada.Agendas parlamentarias was not found in location europe-southwest1; reason: notFound, message: Not found: Table emerald-vigil-438909-s2:BBDD_unificada.Agendas parlamentarias was not found in location europe-southwest1

Location: europe-southwest1
Job ID: 677df6ed-0d6a-49ea-bc0b-7082d8be706e



In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account

SERVICE_ACCOUNT_FILE = '/content/drive/Shareddrives/(02 MAD) Iniciativas - AAPP - JP/Automatización_py/Exportación_BigQuery/emerald-vigil-438909-s2-74505858a60d.json'

credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE)

project_id = 'emerald-vigil-438909-s2'
client = bigquery.Client(credentials=credentials, project=project_id)

dataset_id = 'BBDD_unificada'
table_id = 'Agendas_parlamentarias'
table_ref = f"{project_id}.{dataset_id}.{table_id}"

job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_APPEND'
    write_disposition='WRITE_TRUNCATE',  # caso a tabela nao exista
    autodetect=True,  # caso a tabela nao exista
)

job = client.load_table_from_dataframe(dfag, table_ref, job_config=job_config)
job.result()

print(f" Tabla cargada correctamente: {table_ref}")

✅ Tabla cargada correctamente: emerald-vigil-438909-s2.BBDD_unificada.Agendas_parlamentarias


In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account

SERVICE_ACCOUNT_FILE = '/content/drive/Shareddrives/(02 MAD) Iniciativas - AAPP - JP/Automatización_py/Exportación_BigQuery/emerald-vigil-438909-s2-74505858a60d.json'

credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE)

project_id = 'emerald-vigil-438909-s2'
client = bigquery.Client(credentials=credentials, project=project_id)

dataset_id = 'BBDD_unificada'
table_id = 'Agendas_parlamentarias'
table_ref = f"{project_id}.{dataset_id}.{table_id}"

job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_APPEND'
    #write_disposition='WRITE_TRUNCATE',  # caso a tabela nao exista
    #autodetect=True,  # caso a tabela nao exista
)

job = client.load_table_from_dataframe(dfag_nuevos, table_ref, job_config=job_config)
job.result()

print(f" Tabla cargada correctamente: {table_ref}")

 Tabla cargada correctamente: emerald-vigil-438909-s2.BBDD_unificada.Agendas_parlamentarias


# **Et... voilà!**